In [1]:
import os, sys
stdout = sys.stdout
import ast
from datetime import datetime
import copy
import logging
import time

import matplotlib.pyplot as plt
%matplotlib inline

from astropy import units as u
from astropy.io import ascii
from astropy.io import fits
from astropy.table import Table, unique, Column, join, setdiff
from astropy.coordinates import SkyCoord
import numpy as np
import scipy as sp

from specutils import Spectrum1D
from linetools.spectra.xspectrum1d import XSpectrum1D
from linetools.spectra import utils as ltsu
from matplotlib.backends.backend_pdf import PdfPages
from astropy.cosmology import WMAP9 as cosmo
import pandas as pd

import threading as th

#from pyigm.surveys.dlasurvey import DLASurvey
#from pyigm.surveys.llssurvey import LLSSurvey
#from pyigm.surveys.igmsurvey import GenericIGMSurvey
from pyigm.abssys.lls import LLSSystem
from pyigm.continuum import quasar as pyicq

#from pyigm.fN.fnmodel import FNModel
#from pyigm.fN.constraints import FNConstraint
#from pyigm.fN import tau_eff as teff

from linetools.spectralline import AbsLine
from linetools.isgm.abscomponent import AbsComponent

import random
import glob
import json
import warnings

warnings.filterwarnings("ignore")

# --- --- --- ---

-----------------------------------------------------------
-----------------------------------------------------------
 Install pymc if you want it
-----------------------------------------------------------


In [2]:
from astropy.table import Table
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
import numpy
import copy
import math
import statistics
import os
import glob
from numpy import arange
from pandas import read_csv
import scipy
from scipy.optimize import curve_fit
import multiprocessing
from pyigm.continuum import quasar as pyicq
from astropy import units as u
import pickle
from pylab import figure, cm
from matplotlib.colors import LogNorm

In [3]:
from sklearn.ensemble import AdaBoostRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import HuberRegressor, PoissonRegressor

from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesClassifier, ExtraTreesRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.neural_network import MLPClassifier

In [ ]:
def blockPrint():
    pass
    sys.stdout = open(os.devnull,'w')

# Restore
def enablePrint():
    sys.stdout = stdout

In [ ]:
def prepMockComposite():
    
    global cont
    
    cont = pyicq.get_telfer_spec(zqso=1.0)
    
    global contW
    global contF
    
    contW = cont.wavelength
    contF = cont.flux
    

In [ ]:
def sortNewFile(ii):

    f = open('./mockGeneration/mockGenerator_temp.txt', 'r')
    lines = f.readlines()
    f.close()

    lines = sorted(lines, key=(lambda x: int(x.split()[0])))

    f = open('./mockGeneration/mockGenerator.txt', 'a')
    for l in lines:
        f.write(l)
    f.close()
    
    os.remove('./mockGeneration/mockGenerator_temp.txt')
    
    f = open('./mockGeneration/currentCount.txt', 'w')
    f.write('{}'.format(ii))
    f.close()
        

In [ ]:
def createLLSSpectrum(mainLock, ii, zQSO, rnum, zLLS, mtab, zllsar, tauar, snar, sign, g1): 
    
    tau = np.random.choice(tauar)+((-1.)**np.random.choice(2,1))*np.random.choice(11,1)*.02
    nhi = 17.2+np.log10(tau)

    #load continuum (Telfer)
    thisCont = pyicq.get_telfer_spec(zqso)

    #prepare arrays using GALEX data
    error=mtab['sigma'][rnum]/np.median(mtab['flux'][rnum][g1])
    earr = []
    farr = []
    #originally where error was added to flux - but changed to add error after the LLS
    #so this just creates flux array with values of 1
    for errs in error:
        tmp = np.random.normal(0,abs(errs),1)
        if errs < 0.: 
            tmp = 0.
            farr.append(1.)
        else:
            farr.append(1.)
    sp = XSpectrum1D.from_tuple((mtab['wave'][rnum],np.array(farr)/np.median(np.array(farr)[g1]),error))
    rbc = thisCont.rebin(sp.wavelength)
    #mock flux and sig, before LLS is added
    pllsfl = rbc.flux*sp.flux/np.median(rbc.flux[g1])
    pllsef = sp.sig

    g3 = np.where(np.abs(sp.wavelength.value-2200)<50)
    g4 = np.where(np.abs(mtab['wave'][rnum]-2200)<50)


    #Create LLS Model
    lls = LLSSystem((0.*u.deg,0.*u.deg),zlls,None,NHI=nhi)
    lls.fill_lls_lines(bval=50.*u.km/u.s)
    spec = XSpectrum1D.from_tuple((rbc.wavelength,np.ones(len(rbc.wavelength))))
    model = lls.flux_model(spec)
    smmod = model.box_smooth(3)#smooth abs. lines - this is generic, could make specific to GALEX
    lflux = smmod.flux

    modelflux = rbc.flux*lflux/np.median(rbc.flux[g1])
    xflux = pllsfl*lflux
    #####
    #####
    xfarr = []
    
    # random 0, x: 
    for jj in range(len(error)):
        tmp = np.random.normal(0,abs(error[jj]),1)
        if error[jj] < 0.: 
            tmp = 0.
            xfarr.append(tmp)
        else:
            xfarr.append(tmp+xflux[jj])  #<--- here
            
    ysp=XSpectrum1D.from_tuple((rbc.wavelength,np.array(xfarr),sp.sig))
    #print(np.median(ysp.flux[g3]/ysp.sig[g3]))
    #print(np.median(mtab['flux'][rnum][g4]/np.median(mtab['flux'][rnum][g1])/error[g4]))

    afterLL = np.where(abs(ysp.wavelength - (912.*(1+zlls)-100.)*u.AA)<50.*u.AA)
    beforeLL1 = np.where(abs(ysp.wavelength - (912.*(1+zlls)+200.)*u.AA)<50.*u.AA)
    beforeLL2 = np.where(abs(ysp.wavelength - (912.*(1+zlls)+100.)*u.AA)<50.*u.AA)
    after_ms = np.mean(ysp.sig[afterLL])
    before2_ms = np.mean(ysp.sig[beforeLL2])
    before1_ms = np.mean(ysp.sig[beforeLL1])
    after_mf = np.mean(ysp.flux[afterLL])
    before2_mf = np.mean(ysp.flux[beforeLL2])
    before1_mf = np.mean(ysp.flux[beforeLL1])
    s_nh2 = before2_mf/before2_ms
    s_nh1 = before1_mf/before1_ms
    s_nl = after_mf/after_ms

    hh=np.random.choice(23)
    if hh<10:
        hhs='0'+str(hh)
    else:
        hhs=str(hh)
    mm=np.random.choice(59)
    if mm<10:
        mms='0'+str(mm)
    else:
        mms=str(mm)
    ss=np.random.choice(59)
    if ss<10:
        sss='0'+str(ss)
    else:
        sss=str(ss)
    dd=np.random.choice(89)
    if dd<10:
        dds='0'+str(dd)
    else:
        dds=str(dd)
    am=np.random.choice(59)
    if am<10:
        ams='0'+str(am)
    else:
        ams=str(am)
    aa=np.random.choice(59)
    if aa<10:
        aas='0'+str(aa)
    else:
        aas=str(aa)

    tname='J'+hhs+mms+sss+'.'+str(np.random.choice(9))+\
    np.random.choice(sign)+dds+ams+aas+'.'+str(np.random.choice(9))

    printStr = '{}'.format(ii)
    printStr += ' {}'.format(tname)
    printStr += ' {}'.format(zqso)
    printStr += ' {}'.format(zlls)
    printStr += ' {}'.format(s_nl)
    printStr += ' {}'.format(s_nh2)
    printStr += ' {}'.format(s_nh1)
    printStr += ' {}'.format(tau[0])
    
    printStr += ' [{},'.format(float(ysp.wavelength[0]/u.AA))
    for w in ysp.wavelength[1:-1]:
        x = float(w/u.AA)
        printStr += '{},'.format(x)
    printStr += '{}]'.format(float(ysp.wavelength[-1]/u.AA))
    
    printStr += ' [{},'.format(ysp.flux[0])
    for w in ysp.flux[1:-1]:
        printStr += '{},'.format(w)
    printStr += '{}]'.format(ysp.flux[-1])

    printStr += ' [{},'.format(ysp.sig[0])
    for w in ysp.sig[1:-1]:
        printStr += '{},'.format(w)
    printStr += '{}]'.format(ysp.sig[-1])

    printStr += ' [{},'.format(rbc.flux[0]/np.median(rbc.flux[g1]))
    for w in rbc.flux[1:-1]:
        printStr += '{},'.format(w/np.median(rbc.flux[g1]))
    printStr += '{}]'.format(rbc.flux[-1]/np.median(rbc.flux[g1]))

    printStr += ' [{},'.format(modelflux[0])
    for w in modelflux[1:-1]:
        printStr += '{},'.format(w)
    printStr += '{}]'.format(modelflux[-1])
    
    printStr += '\n'

    mainLock.acquire()
    try:
        f = open('./mockGeneration/mockGenerator_temp.txt', 'a')
        f.write(printStr)
        f.close()
    finally:
        mainLock.release()
                   

In [ ]:
def key_capture_thread():
    global keep_going
    input()
    keep_going = False

In [ ]:
#enablePrint()

glx = Table.read('gm16_sort_20200721.fits')

# --- --- --- ---

##updated on 04.21.2021 - mock spectra created from real sig. data
mtab = glx[glx['z']>0.6]

zqsoar = np.linspace(0.6,2.6,401)
zllsar = np.linspace(0.7,1.8,201)
tauar = np.array([0.2,0.3,0.7,1.0,1.5,2.0,2.5,3.])
snar = np.array([0.5,0.75,1.,1.25,1.5,2.0])

sign=['+','+','+','-']
ii=0
startI = 0
runThis = True
count = -1

printRange = []

mainLock = multiprocessing.Lock()
coreCount = multiprocessing.cpu_count()
processList = []
if runThis:     
    #prepMockComposite()
    mockFile = glob.glob('./mockGeneration/mockGenerator.txt')
    if len(mockFile) == 0:
        f = open('./mockGeneration/mockGenerator.txt', 'w')
        f.write('MockID MockName ZQSO ZLLS S_N_blue S_N_red100 S_N_red200 TAU wave flux sigma sc_flux slls_flux\n')
        f.close()
        ii = 0
        startI = 0
    else:
        f = open('./mockGeneration/currentCount.txt', 'r')
        ii = int(f.readlines()[0])
        f.close()
        startI = ii
        mockTemp = glob.glob('./mockGeneration/mockGenerator_temp.txt')
        if len(mockTemp) > 0:
            os.remove(mockTemp[0])
    f = open('./mockGeneration/currentSession.txt', 'a')
    f.write('{} -- Starting new session at spec {}\n'.format(datetime.now(), ii))
    f.close()
    keep_going = True
    th.Thread(target=key_capture_thread, args=(), name='key_capture_thread', daemon=True).start()
    blockPrint()
    while keep_going:
        for p in processList:
            if p.is_alive() == False:
                p.close()
                processList.remove(p)
        # Launch new processes
        while (len(processList) < coreCount):
            rnum = np.random.choice(len(mtab))
            zqso = mtab['z'][rnum]#np.random.choice(zqsoar)
            zlls = np.random.choice(zllsar)
            g1 = np.where(np.abs(mtab['wave'][rnum]-2500)<50)
            if (zlls < zqso) and (np.median(mtab['flux'][rnum][g1])>0.):
                if (ii-startI)%1000 == 0 and (ii-startI) != 0:
                    for p in processList:
                        p.join()
                    p = multiprocessing.Process(target=sortNewFile, args=(ii, ))
                    p.start()
                    p.join()
                    f = open('./mockGeneration/currentSession.txt', 'a')
                    f.write('{} -- Added {} specs\n'.format(datetime.now(), ii-startI))
                    f.close()
                ii += 1
                p = multiprocessing.Process(target=createLLSSpectrum, args=(mainLock, ii, zqso, rnum, zlls, mtab, zllsar, tauar, snar, sign, g1))
                processList.append(p)
                p.start() 
    for p in processList:
        p.join()
            
    p = multiprocessing.Process(target=sortNewFile, args=(ii, ))
    p.start()
    p.join()
    
    f = open('./mockGeneration/history.txt', 'a')
    f.write('{} -- Added {}\n'.format(datetime.now(), ii-startI))
    f.close()
    
    os.remove('./mockGeneration/currentSession.txt')

for p in processList:
    p.join()    

enablePrint() 
print('Done')

In [ ]:
def generateTextSample():
    
    want = int(input())
    
    f = open('./mockGeneration/mockGenerator.txt', 'r')
    lines = f.readlines()
    f.close()

    f = open('./mockGeneration/mocks_{}.txt'.format(want), 'w')
    f.write(lines[0])
    if want == -1 or want >= len(lines):
        for l in lines[1:]:
            f.write(l)
    else:
        for l in lines[1:want+1]:
            f.write(l)
    f.close()
    del lines
    del f
    
generateTextSample()


In [ ]:
def readFile(file):
    
    f = open(file, 'r')
    lines = f.readlines()
    f.close()
    
    return lines

In [ ]:
def generateFitsTable(data, names, f):
    
    tab = Table(data, names=names)
    tab['ZQSO'].info.format = '4.2f'
    tab['ZLLS'].info.format = '4.2f'
    tab['S_N_blue'].info.format = '4.2f'
    tab['S_N_red100'].info.format = '4.2f'
    tab['S_N_red200'].info.format = '4.2f'
    tab['TAU'].info.format = '4.2f'

    tab.write(f,format='fits',overwrite=True)
    del tab
    

In [ ]:
def createFitsFromText(file):
    
    #lines = readFile(file)

    wl = []
    fl = []
    el = []
    sconfl = []
    sllsfl = []
    zql = []
    zll = []
    snhl1 = []
    snhl2 = []
    snll = []
    tl = []
    inl = []
    namel = []
    
    with open(file) as theFile:
        i = 0
        for l in theFile:
            if i != 0:
                ls = l.split()
                inl.append(int(ls[0]))
                namel.append(ls[1])
                zql.append(float(ls[2]))
                zll.append(float(ls[3]))
                snll.append(float(ls[4]))
                snhl2.append(float(ls[5]))
                snhl1.append(float(ls[6]))
                tl.append(float(ls[7]))
                wl.append(np.array(ast.literal_eval(ls[8])))
                fl.append(np.array(ast.literal_eval(ls[9])))
                el.append(np.array(ast.literal_eval(ls[10])))
                sconfl.append(np.array(ast.literal_eval(ls[11])))
                sllsfl.append(np.array(ast.literal_eval(ls[12])))
            i += 1
    data = [inl,namel,zql,zll,snll,snhl2,snhl1,tl,wl,fl,el,sconfl,sllsfl]
    names = ['MockID','MockName','ZQSO','ZLLS','S_N_blue','S_N_red100','S_N_red200','TAU','wave','flux','sigma','sc_flux','slls_flux']
    
    generateFitsTable(data, names, file[-4:]+'.fits')

    print('Done')

# --- --- --- ---
f = './mockGeneration/mocks_100000.txt'
createFitsFromText(f)

In [8]:
def calculateEMA(lst, smoothing, days):
    
    if days <= 1:
        #print('Invalid period, returning original list.')
        return lst
    
    ema = []
    startSma = []
    for i, item in enumerate(lst):
        if i < days:
            runningEma = item
            if False and i > 0:
                runningEma = (item * (smoothing*2/(1+i))) + (ema[-1] * (1 - (smoothing*2/(1+i))))
            ema.append(item)
        else:
            newEma = (item * (smoothing/(1+days))) + (ema[-1] * (1 - (smoothing/(1+days))))
            ema.append(newEma)
    
    return allAboveValue(ema, -1)


In [9]:
def getPureSmoothed(lst, smoothing):
    
    if smoothing <= 1:
        return lst
    
    up = 0
    down = 0
    if smoothing % 2 == 0:
        up = int(smoothing/2)
        down = int(smoothing/2)
    else:
        up = int(smoothing/2 + 1.0)
        down = int(smoothing/2)
    
    smoothedList = []
    for i, item in enumerate(lst):
        if i >= down and i <= (len(lst) - up):
            smoothedList.append(np.mean(lst[i-down:i+up]))
        elif i < down:
            smoothedList.append(np.mean(lst[:i+up]))
        else:
            smoothedList.append(np.mean(lst[i-down:]))
            
    return smoothedList

In [10]:
def weightedArithmeticMean(flux, sigma):
    
    if len(flux) == 1:
        return flux[0]/sigma[0]
    elif len(flux) == 0:
        return 0.
    
    num = 0.
    den = 0.
    
    lastNum = 0.
    lastDen = 0.001
    
    for i in range(0, len(flux)):
        num += (flux[i]*(1/sigma[i]**2))
        den += (1/sigma[i]**2)
        lastNum = (flux[i]*(1/sigma[i]**2))
        lastDen = (1/sigma[i]**2)
    
    return num/den

In [11]:
def getSigmaSmoothed(lst, smoothing, sigma):
    
    if smoothing <= 1:
        return lst
    
    up = 0
    down = 0
    if smoothing % 2 == 0:
        up = int(smoothing/2)
        down = int(smoothing/2)
    else:
        up = int(smoothing/2 + 1.0)
        down = int(smoothing/2)
    
    smoothedList = []
    j = 0
    while j < len(sigma):
        if sigma[j] >= 0.:
            break
        else:
            j += 1
    
    for i in range(0, j):
        smoothedList.append(weightedArithmeticMean(lst[j-down:j+up], sigma[j-down:j+up]))
        
    for i in range(j, len(lst)):
        if i >= down and i <= (len(lst) - up):
            smoothedList.append(weightedArithmeticMean(lst[i-down:i+up], sigma[i-down:i+up]))
        elif i < down:
            smoothedList.append(weightedArithmeticMean(lst[:i+up], sigma[:i+up]))
        else:
            smoothedList.append(weightedArithmeticMean(lst[i-down:], sigma[i-down:]))
            
    return smoothedList

In [12]:
def combineShelves(shelves, mainFlux, smoothing):
    
    mod = int(smoothing/5)
    
    i = 0
    while i < len(shelves)-1:
        thisShelf = shelves[i]
        nextShelf = shelves[i+1]
        if (nextShelf[0] - thisShelf[-1] <= mod) or ((nextShelf[0] - thisShelf[-1] <= mod*2) and np.median(mainFlux[thisShelf]) <= np.median(mainFlux[nextShelf])):
            for j in range(thisShelf[-1]+1, nextShelf[0]):
                shelves[i].append(j)
            for j in nextShelf:
                shelves[i].append(j)
            del shelves[i+1]
        else:
            i += 1
            
    return shelves

In [13]:
def identifyShelves(ema12, ema26, mainWave, mainFlux, mainSigma, zQSO, startIndex):
    
    shelves = []
    newShelf = []
    
    for i in range(0, len(ema12)):
        check = mainWave[startIndex+i] <= 950*(1+zQSO) and mainWave[startIndex+i] >= 900*(1.6)
        if len(newShelf) > 0:
            check = mainWave[newShelf[0]] <= 950*(1+zQSO) and mainWave[newShelf[0]] >= 900*(1.6)
        if (mainFlux[i] != 0. and mainSigma[i] >= 0) and ema12[i] > ema26[i] and check:
            #print('here', ema12[i], ema26[i], i+startIndex)
            newShelf.append(i+startIndex)
        else:
            if len(newShelf) > 2:
                #print('FOUND A SHELF FROM', newShelf[0], 'to', newShelf[-1])
                shelves.append(newShelf)
            newShelf = []
    
    if len(newShelf) > 2:
        #print('FOUND A SHELF FROM', newShelf[0], 'to', newShelf[-1])
        shelves.append(newShelf)
    
    #print('Found', len(shelves), 'potential shelves')
    
    return shelves

In [14]:
def initialCompositePrep(z=1.0):
    
    pre_cont = pyicq.get_telfer_spec(zqso=z)
    wave = []
    flux = []
    
    for i, psw in enumerate(pre_cont.wavelength):
        #print(pre_cont.flux[i])
        wave.append(float(psw/u.AA))
        flux.append(float(pre_cont.flux[i]))
    
    return wave, flux, pre_cont


In [79]:
def prepCompositeSpectrum(comp, flux, wave, zQSO, zLLS, tau, modifyWave=True):
    
    pre_cont = comp
    
    ratioCut = None
    if 950*(1+zQSO) <= 2700:
        ratioCut = 950*(1+zQSO)
    else:
        ratioCut = 2700
    
    waveMod = 0
    if modifyWave:
        waveMod = 912.*(1+zQSO) - 912.*(1+1)
        
    perfectMean = []
    perfectSpectrumWave = []
    perfectSpectrumFlux = []
    recoveryWave = []
    recoveryFlux = []
    postLLSWave = []
    postLLSFlux = []
    for i, psw in enumerate(pre_cont[0]):
        #print(pre_cont.flux[i])
        newWave = psw+waveMod
        if 1400 <= newWave <= 2800:
            perfectSpectrumWave.append(newWave)
            if newWave <= 912*(1+zLLS):
                recoveryWave.append(newWave)
                recoveryFlux.append(float(pre_cont[1][i]*np.exp(-tau*(psw/(912.*(1+zLLS)))**3)))
            else: 
                postLLSWave.append(newWave)
                postLLSFlux.append(float(pre_cont[1][i]))
            perfectSpectrumFlux.append(float(pre_cont[1][i]))
            if newWave >= ratioCut:
                perfectMean.append(float(pre_cont[1][i]))
    
    
    zone = flux[wave >= ratioCut]
    meanRatio = np.median(zone)/np.median(perfectMean)
    #print(smoothing, meanRatio)
    newPSF = []
    for i, f in enumerate(perfectSpectrumFlux):
        newPSF.append(f*meanRatio)
    
    nrf = []
    for f in recoveryFlux:
        nrf.append(f*meanRatio)
    recoveryFlux = nrf
    
    '''normFlux = []
    normWave = []
    mod = np.median(flux)
    for i, f in enumerate(flux):
        for j, w in enumerate(perfectSpectrumWave):
            if wave[i] <= w:
                #print(wave[i], w)
                normFlux.append(f-newPSF[j]+(mod)*2)
                normWave.append(wave[i])
                break'''
    
    return perfectSpectrumWave, newPSF, recoveryWave, recoveryFlux, meanRatio, postLLSWave, postLLSFlux, ratioCut

In [70]:
def normalizeResolution(zndex, wave1, flux1, wave2, flux2, start, end):
    
    if len(wave1) <= 1 or len(wave2) <= 1:
        return None
    
    diffMod = np.mean(flux2[-5:])-np.mean(flux1[-5:])
    diffMod = 0
    
    newFlux1 = []
    for f in flux1:
        newFlux1.append(f+diffMod)
    flux1 = newFlux1
    
    res1 = (wave1[-1]-wave1[0])/len(wave1)
    res2 = (wave2[-1]-wave2[0])/len(wave2)
    
    if res1 == 0. or res2 == 0.:
        print(zndex)
        print(len(wave1), len(wave2))
        print(wave1[-1]-wave1[0], wave2[-1]-wave2[0])
        print(res1, res2)
    
    lowWave = None
    lowFlux = None
    highWave = None
    highFlux = None
    lowRes = None
    highRes = None
    
    if res1 < res2:
        # Wave 1 is higher resolution
        lowWave = wave2
        lowFlux = flux2
        highWave = wave1
        highFlux = flux1
        lowRes = res2
        highRes = res1
    else:
        # Wave 2 is higher resolution
        lowWave = wave1
        lowFlux = flux1
        highWave = wave2
        highFlux = flux2
        lowRes = res1
        highRes = res2
    
    newWave = []
    newFlux = []
    newLWave = []
    newLFlux = []
    '''nextMatch = len(highWave)-1
    lowIndex = len(lowWave)-1
    mod = 0.
    for i in range(len(highWave)-1, -1, -1):
        if highWave[i] >= end and lowIndex >= 0:
            if i == nextMatch:
                
                lowIndex -= 1
                mod += (lowRes/highRes) - int(lowRes/highRes)
                nextMatch -= int(lowRes/highRes)+int(mod)
                mod = mod - int(mod)
                if nextMatch <= 0:
                    nextMatch = 0
        else:
            break'''
    
    i = len(highWave)-1
    j = len(lowWave)-1
    while j >= 0:
        temp = []
        while highWave[i] > lowWave[j] and i >= 0:
            temp.append(highFlux[i])
            i -= 1
        newWave.append(lowWave[j])
        newFlux.append(np.mean(temp))
        newLWave.append(lowWave[j])
        newLFlux.append(lowFlux[j]) 
        j -= 1
                
    return list(reversed(newWave)), list(reversed(newFlux)), list(reversed(newLWave)), list(reversed(newLFlux))

In [56]:
def chi2(l1, l2):
    
    c2 = scipy.stats.chisquare(l1, l2)
    
    return c2

In [57]:
def estimateTau(pShelf, rawFlux):
    
    modFlux = np.array(allAboveValue(rawFlux, 0.001))
    
    #eTau = np.log((max(modFlux[pShelf]))/(min(modFlux[pShelf])))
    eTau = np.log((np.percentile(modFlux[pShelf], 95))/(np.percentile(modFlux[pShelf], 5)))
    rawTau = eTau
    if eTau > 3.:
        eTau = 3.
    elif eTau < 0.:
        eTau = 0.
    
    return eTau, rawTau

In [58]:
def allAboveValue(lst, value):
    newList = []
    for item in lst:
        if item < value:
            newList.append(value)
        else:
            newList.append(item)
            
    return newList

In [59]:
def austinSmooth(l, s, e):
    
    sigmaL = getSigmaSmoothed(l, s, e)
    emaL = calculateEMA(l, 2, 26)
    
    finalL = []
    for i in range(0, len(l)):
        finalL.append((sigmaL[i]+emaL[i])/2)
        
    return finalL
    

In [60]:
def normalizedContinuum(fluxOver, fluxUnder, waveOver, waveUnder, index=-1):
    
    length = len(fluxOver)
    if len(fluxOver) != len(fluxUnder):
        print('Error at', index, len(fluxOver), len(fluxUnder))
        length = len(fluxUnder)
        
    normalizedFlux = []
    normalizedWave = []
    for i in range(0, length):
        normalizedFlux.append(fluxOver[i]/fluxUnder[i])
        normalizedWave.append(waveOver[i])
        
    return normalizedFlux, normalizedWave


In [61]:
class Shelf():
    
    def __init__(self, cont, stats, parent, score):
        self.cont = cont
        self.stats = stats
        self.parent = parent
        self.score = score
        self.children = None
        self.height = -1
        if parent != None:
            self.height = self.parent.height + 1
        else:
            self.height = 0
            
    def printStructure(self):
        if self.children != None:
            for c in self.children:
                c.printStructure()
        else:
            print('{} -> {}'.format(self.cont[0], self.cont[-1]))
            return 0

In [62]:
def prepFiles(f):
    
    dir1 = './automatedOutput/{}'.format(f)
    if not os.path.isdir(dir1):
        os.mkdir(dir1)
    if not os.path.isdir(dir1+'/tauShelves'):
        os.mkdir(dir1+'/tauShelves')
    if not os.path.isdir(dir1+'/definedShelves'):
        os.mkdir(dir1+'/definedShelves')
    
    img = glob.glob('./automatedOutput/{}/**/*.png'.format(f), recursive=True)
    for i in img:
        os.remove(i)
        
    txt = glob.glob('./automatedOutput/{}/**/*.txt'.format(f), recursive=True)
    for t in txt:
        os.remove(t)
        
    csv = glob.glob('./automatedOutput/{}/**/*.csv'.format(f), recursive=True)
    for c in csv:
        os.remove(c)
        
    dirs = glob.glob('./automatedOutput/{}/**/*.index'.format(f))
    for d in dirs:
        os.rmdir(d)
        
    return 0


In [63]:
def returnTauStats(modFlux, s, nonZeroSigma, flux, wave, sigma, m, closestShelf, index, lines):
    
    this = []
        
    this.append(min(modFlux[s]))
    this.append(max(modFlux[s]))
    this.append(max(modFlux[s])-min(modFlux[s]))
    this.append((max(modFlux[s])-min(modFlux[s]))/(min(modFlux[s])+0.001))
    this.append((max(modFlux[s])-min(modFlux[s]))/len(s))
    this.append(sum(value <= 0 for value in lines[-1][s[0]:s[-1]+1]))
    this.append(len(s))
    this.append(np.mean(nonZeroSigma[s]))

    signalNoise = np.mean(flux[wave > 2050])/np.mean(sigma[wave > 2050])
    this.append(signalNoise) #signal to noise
    this.append(wave[s[0]]) #start
    this.append(wave[s[-1]]) #end

    tauEstimate = estimateTau(s, allAboveValue(flux, 0.001))
    this.append(tauEstimate[0])
    this.append(tauEstimate[1])
    
    return np.array(this).reshape(1, -1)


In [64]:
def printStats(dir1, zndex, shelves, lines, m, wave, sigma, flux, smoothing, density, shelfFile, mainLock, model, comp, columns={}):
    
    modFlux = np.array(lines[-1])
    shelfDefFile = shelfFile[:-4] + '_def.csv'
    nonZeroSigma = np.array(allAboveValue(sigma, 0.001))
    
    longestShelf = 0
    highestMin = 0.
    longestChain = 0
    largestShelf = 0.
    highestSigma = 0.
    highestStDev = 0.
    highestADelta = 0.
    percentDelta = 0.
    for s in shelves:
        if len(s) > longestShelf:
            longestShelf = len(s)
        if min(lines[-1][s[0]:s[-1]+1]) > highestMin:
            highestMin = min(lines[-1][s[0]:s[-1]+1])
        if max(lines[-1][s[0]:s[-1]+1]) - min(lines[-1][s[0]:s[-1]+1]) > largestShelf:
            largestShelf = max(lines[-1][s[0]:s[-1]+1]) - min(lines[-1][s[0]:s[-1]+1])
        if np.mean(nonZeroSigma[s[0]:s[-1]+1]) > highestSigma:
            highestSigma = np.mean(nonZeroSigma[s[0]:s[-1]+1])
        if sum(value <= 0 for value in lines[-1][s[0]:s[-1]+1]) > longestChain:
            longestChain = sum(value <= 0 for value in lines[-1][s[0]:s[-1]+1])
        
        thisDelta = (max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(s[-1]-s[0])
        if thisDelta > highestADelta:
            highestADelta = (max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(s[-1]-s[0])
        if (max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(0.01+max(lines[-1][s[0]:s[-1]+1])) > percentDelta:
            percentDelta = (max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(0.01+max(lines[-1][s[0]:s[-1]+1]))

    #REDSHIFT??

    if highestMin == 0.0:
        highestMin = 0.1
    if largestShelf == 0.0:
        largestShelf = 0.1
    if longestShelf == 0:
        longestShelf = 1
    if longestChain == 0:
        longestChain = 1
    if highestADelta == 0.0:
        highestADelta = 0.1
    if highestSigma == 0.0:
        highestSigma = 1.
    if percentDelta == 0.0:
        percentDelta = 0.1
    
    closestShelf = (None, None)
    realTau = None
    if columns['tau'] != None and columns['zlls'] != None:
        realTau = m[columns['tau']]
        closestShelf = (9999999, -1)
        for index, s in enumerate(shelves):
            if closestShelf[0] != 0.:
                if (wave[s[0]] <= 912.*(1+m[columns['zlls']]) <= wave[s[int(len(s)/2)]]):
                    closestShelf = (0., index)
                elif wave[s[0]] >= 912.*(1+m[columns['zlls']]) and (abs(912.*(1+m[columns['zlls']]) - wave[s[0]]) < closestShelf[0]):
                    closestShelf = (abs(912.*(1+m[columns['zlls']]) - wave[s[0]]), index)    
        
    scoring = []
    tau = []
    for index, s in enumerate(shelves):
        
        this = []
        
        this.append(((max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(0.01+max(lines[-1][s[0]:s[-1]+1])))/percentDelta)
        this.append(((max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(s[-1]-s[0]))/highestADelta)
        this.append(sum(value <= 0 for value in lines[-1][s[0]:s[-1]+1])/longestChain)
        this.append((max(lines[-1][s[0]:s[-1]+1]) - min(lines[-1][s[0]:s[-1]+1]))/(largestShelf))
        this.append((min(lines[-1][s[0]:s[-1]+1]))/highestMin)
        this.append(len(s)/longestShelf)
        this.append(np.mean(nonZeroSigma[s[0]:s[-1]+1])/highestSigma)
        
        percentile = -1.
        if len(shelves) > 1:
            percentile = np.mean(this)
        else:
            percentile = 1.0

        this = []
        
        this.append(((max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(0.01+max(lines[-1][s[0]:s[-1]+1]))))
        this.append(((max(lines[-1][s[0]:s[-1]+1])-min(lines[-1][s[0]:s[-1]+1]))/(s[-1]-s[0])))
        this.append(sum(value <= 0 for value in lines[-1][s[0]:s[-1]+1]))
        this.append(max(lines[-1][s[0]:s[-1]+1]) - min(lines[-1][s[0]:s[-1]+1]))
        this.append(min(lines[-1][s[0]:s[-1]+1]))
        this.append(len(s))
        this.append(np.mean(nonZeroSigma[s[0]:s[-1]+1]))
        this.append(percentile)
        
        signalNoise = np.mean(flux[wave > 2050])/np.mean(sigma[wave > 2050])
        this.append(signalNoise) #signal to noise
        this.append(wave[s[0]]) #start
        this.append(wave[s[-1]]) #end
        
        tauPrediction = estimateTau(s, flux)
        tau.append(tauPrediction)
        minF = 9999999
        minI = 0
        maxF = 0
        maxI = 0
        for j, f in enumerate(modFlux[s]):
            if f <= minF:
                minI = j
                minF = f
            if f >= maxF:
                maxI = j
                maxF = f
        zLLS = (wave[minI+s[0]]/912.)-1.
        perfectSpectrumWave, perfectSpectrumFlux, rw, rf, meanRatio, _, _, _ = prepCompositeSpectrum(comp, flux, wave, m[columns['zqso']], zLLS, tauPrediction[0], modifyWave=False)
        
        #shelf
        nppsw = np.array(perfectSpectrumWave)
        nppsf = np.array(perfectSpectrumFlux)
        pswz = nppsw[(nppsw >= wave[minI+s[0]]) & (nppsw <= wave[maxI+s[0]])]
        psfz = nppsf[(nppsw >= wave[minI+s[0]]) & (nppsw <= wave[maxI+s[0]])]
        end = wave[s[0]]
        normed = normalizeResolution(zndex, pswz, psfz, wave[minI+s[0]:maxI+s[0]+1], flux[minI+s[0]:maxI+s[0]+1], -1, end)
        if normed != None:
            chi2Fit = chi2(normed[3], normed[1])
            this.append(chi2Fit[0]) # chisq stat
            this.append(chi2Fit[1]) # p-value
            '''plt.plot(normed[0], normed[1], lw=3, label='comp zone')
            plt.plot(normed[2], normed[3], lw=3, label='real zone')
            plt.legend()
            plt.title('{} - {}'.format(wave[minI+s[0]], wave[maxI+s[0]]))
            plt.ylim(-0.1, 10)
            plt.show()
            plt.close()'''
        else:
            this.append(-99)
            this.append(-99)
        
        #recovery
        end = wave[minI+s[0]]-400.
        normed = normalizeResolution(zndex, rw, rf, wave[:minI+s[0]+1], flux[:minI+s[0]+1], -1, end)
        if normed != None:
            chi2Fit = chi2(normed[3], normed[1])
            this.append(chi2Fit[0]) # chisq stat
            this.append(chi2Fit[1]) # p-value
            '''plt.plot(nppsw[nppsw > normed[0][0]], nppsf[nppsw > normed[0][0]], label='composite post-LLS')
            plt.plot(wave, flux, label='real', color='b', alpha=0.3)
            plt.plot(normed[0], normed[1], lw=3, label='comp recovery')
            plt.plot(normed[2], normed[3], lw=3, label='real recovery')
            plt.legend()
            plt.title(meanRatio)
            plt.ylim(-1, 5)
            plt.show()
            plt.close()'''
        else:
            this.append(-99)
            this.append(-99)

        shelfDensity = np.median(density[s])/max(density) #density
        this.append(shelfDensity)
        this.append(int(index == closestShelf[-1]))
        
        scoring.append(this)
        
    for i, s in enumerate(shelves):
        tauPrediction = tau[i]
        printStr = '{}, {}, {}, {}, {}'.format(smoothing+i, i, tauPrediction[0], smoothing, m[columns['zqso']]) #smoothing, shelfNumber, tauPrediction 
        for j in scoring[i][:-1]:
            if math.isnan(j):
                j = -99.
            if math.isinf(j):
                j = 9999999
            printStr += ', {}'.format(j) # percentHeight, averageSlope, zeroChain, height, min, shelfLength, sigmaMean, averagePercentile, signalToNoise, start, end, recoveryFit, shelfDensity, trueShelf
        print(printStr, file=open(shelfFile, 'a'))
        print('{}, {}, {}'.format(smoothing+i, s[0], len(s)), file=open(shelfDefFile, 'a'))
        
    mainLock.acquire()
    try:
        for i, s in enumerate(shelves):
            tauPrediction = tau[i]
            printStr = '{}, {}, {}, {}, {}, {}'.format(m[columns['id']], smoothing+i, i, tauPrediction[0], smoothing, m[columns['zqso']]) #smoothing, shelfNumber, tauPrediction 
            for j in scoring[i]:
                if math.isnan(j):
                    j = -99.
                if math.isinf(j):
                    j = 9999999
                printStr += ', {}'.format(j) # percentHeight, averageSlope, zeroChain, height, min, shelfLength, sigmaMean, averagePercentile, signalToNoise, start, end, recoveryFit, shelfDensity, trueShelf
            print(printStr, file=open(dir1+'/allShelves.csv', 'a'))
            if scoring[i][-1] == 1:
                print('{}, {}, {}, {}, {}, {}'.format(zndex, smoothing, wave[s[0]], tauPrediction[0], tauPrediction[1], realTau), file=open('{}/tauShelves/shelves.csv'.format(dir1), 'a'))
    finally:
        mainLock.release()
    

In [65]:
def printTauStats(dir1, zndex, shelves, lines, m, wave, sigma, flux, smoothing, density, shelfFile, mainLock):
    
    modFlux = np.array(lines[-1])
    shelfDefFile = shelfFile[:-4] + '_def.csv'
    nonZeroSigma = np.array(allAboveValue(sigma, 0.001))
        
    '''print('highestMin', highestMin)
    print('largestShelf', largestShelf)
    print('longestShelf', longestShelf)
    print('longestChain', longestChain)
    print('highestADelta', highestADelta)
    print('highestSigma', highestSigma)'''
    
    closestShelf = (9999999, -1)
    for index, s in enumerate(shelves):
        if closestShelf[0] != 0.:
            if (wave[s[0]] <= 912.*(1+m['ZLLS']) <= wave[s[-1]]):
                closestShelf = (0., index)
            elif (abs(912.*(1+m['ZLLS']) - wave[s[0]]) < closestShelf[0]):
                closestShelf = (abs(912.*(1+m['ZLLS']) - wave[s[0]]), index)    
        
    scoring = []
    for index, s in enumerate(shelves):

        this = []
        
        this.append(min(modFlux[s]))
        this.append(max(modFlux[s]))
        this.append(max(modFlux[s])-min(modFlux[s]))
        this.append((max(modFlux[s])-min(modFlux[s]))/(min(modFlux[s])+0.001))
        this.append((max(modFlux[s])-min(modFlux[s]))/len(s))
        this.append(sum(value <= 0 for value in lines[-1][s[0]:s[-1]+1]))
        this.append(len(s))
        this.append(np.mean(nonZeroSigma[s]))
        
        signalNoise = np.mean(flux[wave > 2050])/np.mean(sigma[wave > 2050])
        this.append(signalNoise) #signal to noise
        this.append(wave[s[0]]) #start
        this.append(wave[s[-1]]) #end
        
        tauEstimate = estimateTau(s, allAboveValue(flux, 0.001))
        this.append(tauEstimate[0])
        this.append(tauEstimate[1])
        this.append(m['TAU'])
        this.append(int(index == closestShelf[-1]))
        
        scoring.append(this)
        
    mainLock.acquire()
    try:
        for i, s in enumerate(shelves):
            if scoring[i][-1] == 1:
                printStr = '{}, {}, {}, {}'.format(zndex, smoothing+i, i, smoothing) #smoothing, shelfNumber, tauPrediction 
                for j in scoring[i][:-1]:
                    if math.isnan(j):
                        j = -99.
                    printStr += ', {}'.format(j) # percentHeight, averageSlope, zeroChain, height, min, shelfLength, sigmaMean, averagePercentile, signalToNoise, start, end, shelfDensity, trueShelf
                print(printStr, file=open('{}/tauShelves/shelves.csv'.format(dir1), 'a'))
    finally:
        mainLock.release()

In [81]:
def prepModel(i, dir1, currentDir, mockSlice, m, smoothings, mainLock, model, columns={}):
    
    realTau = None
    realLLS = None
    if columns['tau'] != None and columns['zlls'] != None:
        realTau = m[columns['tau']]
        realLLS = m[columns['zlls']]
        
    compWave, compFlux, _ = initialCompositePrep(z=m[columns['zqso']])
    
    comp = (compWave, compFlux)
    
    if len(columns) == 0:
        print('No specified columns')
        return 0
    
    plot = True
    
    shelfFile = '{}/shelves.csv'.format(currentDir)
    shelfDefFile = shelfFile[:-4] + '_def.csv'
    
    print('shelfID, shelfNumber,  tauPrediction, smoothing, zqso, percentHeight, averageSlope, zeroChain, height, min, shelfLength, sigmaMean, averagePercentile, signalToNoise, start, end, shelf_chiSq, shelf_p-value, recover_chiSq, recover_p-value, shelfDensity', file=open('{}/shelves.csv'.format(currentDir), 'a'))
    print('shelfID, start, length', file=open(shelfDefFile, 'a'))
    
    mainWave = np.array(m[columns['wave']][(m[columns['wave']] >= 1400) & (m[columns['wave']] <= 2800)])
    mainFlux = np.array(m[columns['flux']][(m[columns['wave']] >= 1400) & (m[columns['wave']] <= 2800)])*columns['fluxMod']
    mainSigma = np.array(m[columns['sigma']][(m[columns['wave']] >= 1400) & (m[columns['wave']] <= 2800)])*columns['fluxMod']
    
    yMax = max(m[columns['flux']][(m[columns['wave']] >= 1750) & (m[columns['wave']] <= 2050)])*columns['fluxMod']*1.1
    
    psw, psf, _, _, _, _, _, ratioCut = prepCompositeSpectrum(comp, mainFlux, mainWave, m[columns['zqso']], 1, 1, modifyWave=False)
    
    headShelf = Shelf([-101, -100, -99], {}, None, 0)
    
    allShelves = []
    currentShelves = [list(range(0, len(mainWave)))]
    averageGuess = []
    tauEstimate = []
    newRawTau = []
    allSmoothings = []
    densityLevels = []
    density = [0]*len(mainWave)
    
    #parentShelf = headShelf
    #headShelf.printStructure()
    
    for s in smoothings:
        
        newShelves = []
        
        smoothedSection = np.array(getSigmaSmoothed(mainFlux, s, mainSigma))
        
        ema12 = np.array(calculateEMA(smoothedSection, 2, 12))
        ema26 = np.array(calculateEMA(smoothedSection, 2, 26))
        
        allSmoothings.append(ema12)
        
        for cs in currentShelves:
            startIndex = cs[0]
            currentEMA12 = ema12[cs]
            currentEMA26 = ema26[cs]
            potentialShelves = identifyShelves(currentEMA12, currentEMA26, mainWave, mainFlux, mainSigma, m[columns['zqso']], startIndex)
            potentialShelves = combineShelves(potentialShelves, mainFlux, s)
            for ps in potentialShelves:
                for f in ps:
                    density[f] += 1
                allShelves.append(ps)
                newShelves.append(ps)
        
        #print(len(newShelves))
        if len(newShelves) > 0:
            printStats(dir1, i, newShelves, [list(smoothedSection)], m, mainWave, mainSigma, smoothedSection, s, np.array(density), shelfFile, mainLock, model, comp, columns)
        
        if plot:
            figure(figsize=(8, 6), dpi=80)
            plt.plot(m[columns['wave']], m[columns['flux']], alpha=0.25)
            plt.plot(m[columns['wave']], m[columns['sigma']], alpha=0.25, color='orange')
            plt.plot(mainWave, ema12, lw=2)
            plt.plot(mainWave, ema26, lw=2)
            for ns in newShelves:
                plt.plot(mainWave[ns], ema26[ns], lw=5)
                
        pWave = None
        pFlux = None
        if len(newShelves) > 0:
            chi2List = []
            for ns in newShelves:
                currentRange = list(range(ns[0], ns[int(len(ns)/2)]))
                predAA = mainWave[ns[0] + ema12[currentRange].argmin(axis=0)]
                #print('AA: ', predAA)
                predZ = (predAA/912.)-1
                #print('Z:  ', predZ)
                tauE = estimateTau(ns, ema12)[1]
                #print('TAU:', tauE)
            newShelves = sorted(newShelves, key=lambda x: -1*abs(len(x)*(ema12[x[0]]-ema12[x[-1]])))
            currentRange = list(range(newShelves[0][0], newShelves[0][int(len(newShelves[0])/2)]))
            averageGuess.append(newShelves[0][0] + ema12[currentRange].argmin(axis=0))
            tauEstimate.append(estimateTau(newShelves[0], ema12)[0])
            newRawTau.append(estimateTau(newShelves[0], ema12)[1])
        
        if plot:
            for fi, f in enumerate(mainFlux):
                if mainSigma[fi] <= 0:
                    plt.vlines(mainWave[fi], -1, yMax, lw=2, color='red')
            if realLLS != None:
                plt.vlines(912*(1+m[columns['zlls']]), -1, yMax, lw=5, color='purple', ls='--')
            plt.vlines(912*(1+m[columns['zqso']]), -1, yMax, lw=5, color='red', ls='--')
            plt.xlim(1400, 2800)
            plt.ylim(-1, 10)
            plt.savefig(dir1 + '/definedShelves/match_{}.index/smoothingShelves_{}.png'.format(m[columns['id']], s))
            plt.close()
        
        #currentShelves = newShelves
    
    normed = normalizeResolution(i, psw, psf, mainWave, mainFlux, -1, 0)
    
    if plot:
        plt.figure(figsize=(16, 10))
        plt.plot(mainWave, mainFlux, color='b', label='flux', alpha=0.2)
        plt.plot(mainWave, allAboveValue(calculateEMA(getPureSmoothed(mainFlux, 5), 2, 12), 0), color='b', label='pure smoothed flux')
        plt.plot(mainWave, allAboveValue(calculateEMA(getSigmaSmoothed(mainFlux, 5, mainSigma), 2, 12), 0), color='r', label='sigma smoothed flux')
        plt.plot(mainWave, mainSigma, color='orange', label='sigma')
        plt.plot(psw, psf, color='k', label='composite')
        if normed != None:
            plt.plot(normed[0], normed[1], color='purple', lw=5, alpha=0.3)
        if realLLS != None:
            plt.vlines(912*(1+m[columns['zlls']]), -1, yMax, lw=5, color='purple', ls='--', label='zlls')
        plt.vlines(912*(1+m[columns['zqso']]), -1, yMax, lw=5, color='red', ls='--', label='zqso')
        plt.vlines(ratioCut, -1, 10, label='ratioCut-{}'.format(ratioCut))
        plt.xlim(1400, 2800)
        plt.ylim(-1, 10)
        plt.legend()
        plt.savefig(dir1 + '/definedShelves/match_{}.index/continuum.png'.format(m[columns['id']]))
        plt.close()
        
        plt.figure(figsize=(16, 10))
        plt.plot(mainWave, [1]*len(mainFlux), color='k', ls=':')
        plt.plot(mainWave, getPureSmoothed(mainFlux, 30), color='b', label='flux', alpha=0.2)
        if normed != None:
            resCompare = dir1+'/definedShelves/match_{}.index/resCompare.csv'.format(m[columns['id']])
            print('highRes,lowRes', file=open(resCompare, 'w'))
            for resI in range(0, len(normed[0])):
                print('{},{}'.format(normed[0][resI], normed[2][resI]), file=open(resCompare, 'a'))
            normedFlux, normedWave = normalizedContinuum(normed[3], normed[1], normed[2], normed[0], m[columns['id']])
            plt.plot(normedWave, getPureSmoothed(normedFlux, 30), color='k', label='normalized flux')
        if realLLS != None:
            plt.vlines(912*(1+m[columns['zlls']]), -3, 3, lw=5, color='purple', ls='--', label='zlls')
        plt.vlines(912*(1+m[columns['zqso']]), -3, 3, lw=5, color='red', ls='--', label='zqso')
        plt.xlim(1400, 2800)
        plt.ylim(0, 2)
        plt.legend()
        plt.savefig(dir1 + '/definedShelves/match_{}.index/normalizedContinuum.png'.format(m[columns['id']]))
        plt.close()
    

In [67]:
def prepTauModel(i, dir1, mockSlice, m, smoothings, mainLock):
        
    shelfFile = '{}/tauShelves/shelves.csv'.format(dir1)
    shelfDefFile = shelfFile[:-4] + '_def.csv'
    
    #print('shelfID, shelfNumber, smoothing, percentHeight, averageSlope, zeroChain, height, min, shelfLength, sigmaMean, averagePercentile, signalToNoise, start, end, shelfDensity', file=open('{}/shelves.csv'.format(currentDir), 'a'))
    #print('shelfID, start, length', file=open(shelfDefFile, 'a'))
    
    mainWave = np.array(m['wave'][(m['wave'] >= 1400) & (m['wave'] <= 2800)])
    mainFlux = np.array(m['flux'][(m['wave'] >= 1400) & (m['wave'] <= 2800)])
    mainSigma = np.array(m['sigma'][(m['wave'] >= 1400) & (m['wave'] <= 2800)])
    
    headShelf = Shelf([-101, -100, -99], {}, None, 0)
    
    allShelves = []
    currentShelves = [list(range(0, len(mainWave)))]
    averageGuess = []
    tauEstimate = []
    newRawTau = []
    allSmoothings = []
    densityLevels = []
    density = [0]*len(mainWave)
    
    #parentShelf = headShelf
    #headShelf.printStructure()
    
    for s in smoothings:
        
        newShelves = []
        
        smoothedSection = np.array(getPureSmoothed(mainFlux, s))
        
        ema12 = np.array(calculateEMA(smoothedSection, 2, 12))
        ema26 = np.array(calculateEMA(smoothedSection, 2, 26))
        
        allSmoothings.append(ema12)
        
        for cs in currentShelves:
            startIndex = cs[0]
            currentEMA12 = ema12[cs]
            currentEMA26 = ema26[cs]
            potentialShelves = identifyShelves(currentEMA12, currentEMA26, mainWave, m['ZQSO'], startIndex)
            potentialShelves = combineShelves(potentialShelves, mainFlux, s)
            for ps in potentialShelves:
                for f in ps:
                    density[f] += 1
                allShelves.append(ps)
                newShelves.append(ps)
        
        for ns in newShelves:
            pass
            #plt.plot(mainWave[ns], ema26[ns], lw=5)
        
        if len(newShelves) > 0:
            printTauStats(dir1, i, newShelves, [list(smoothedSection)], m, mainWave, mainSigma, smoothedSection, s, np.array(density), shelfFile, mainLock)
        
        pWave = None
        pFlux = None
        if len(newShelves) > 0:
            chi2List = []
            for ns in newShelves:
                currentRange = list(range(ns[0], ns[int(len(ns)/2)]))
                predAA = mainWave[ns[0] + ema12[currentRange].argmin(axis=0)]
                #print('AA: ', predAA)
                predZ = (predAA/912.)-1
                #print('Z:  ', predZ)
                tauE = estimateTau(ns, ema12)[1]
                #print('TAU:', tauE)
            newShelves = sorted(newShelves, key=lambda x: -1*abs(len(x)*(ema12[x[0]]-ema12[x[-1]])))
            currentRange = list(range(newShelves[0][0], newShelves[0][int(len(newShelves[0])/2)]))
            averageGuess.append(newShelves[0][0] + ema12[currentRange].argmin(axis=0))
            tauEstimate.append(estimateTau(newShelves[0], ema12)[0])
            newRawTau.append(estimateTau(newShelves[0], ema12)[1])
        
        #plt.vlines(912*(1+m['ZLLS']), -1, 5, lw=5, color='purple', ls='--')
        #plt.vlines(912*(1+m['ZQSO']), -1, 5, lw=5, color='red', ls='--')
        #plt.xlim(1400, 2800)
        #plt.ylim(-1, 5)
        #plt.savefig('./automatedOutput/modelPrep/{}.index/smoothingShelves_{}.png'.format(i, s))
        #plt.close()
        
        #currentShelves = newShelves
    
    #print(averageGuess)
    #print(tauEstimate)
    #print(np.median(tauEstimate), np.median(tauEstimate)+1.0)
    one = int(len(tauEstimate)/2)
    two = one+1
    
    if one >= len(tauEstimate) or two >= len(tauEstimate) or one < 0:
        one = len(tauEstimate)-2
        two = len(tauEstimate)-1
    
    #prediction = (mainWave[averageGuess[two]]+mainWave[averageGuess[one]])/2
    if len(tauEstimate) > 0:
        prediction = mainWave[averageGuess[one]]
        tauPrediction = tauEstimate
    else:
        prediction = 912*(1+m['ZQSO'])
        tauPrediction = -1
    if prediction >= 912*(1+m['ZQSO']):
        prediction = 912*(1+m['ZQSO'])
    printStr = '{}, {}, {}, {}'.format(i, prediction, prediction-912*(1+m['ZLLS']), tauPrediction, m['TAU'])
    #predictions.append(prediction-912*(1+m['ZLLS']))
    tau.append(m['TAU'])
    #tauPreds.append((tauEstimate[one]+tauEstimate[two])/2)
    #rawTau.append((newRawTau[one]+newRawTau[two])/2)
    #tauPreds.append(tauEstimate[one])
    #rawTau.append(newRawTau[one])

In [ ]:
makeTauModel = False
makeNewModel = True

#fitsFile = 'glx_new_llsmocks2.fits'
#fitsFile = 'austinsMocks_1000.fits'
#fitsFile = 'austinsMocks_25000.fits'
#fitsFile = 'austinsMocks_modelSet_50000.fits'
fitsFile = './gm16_sort_20200721.fits'
f = fitsFile

prepFiles(f)
dir1 = './automatedOutput/{}'.format(f)

mocks = Table.read(f)
mockSlice = mocks
cols = list(mocks.columns)
print('matchIndex, shelfID, shelfNumber, tauPrediction, smoothing, zqso, percentHeight, averageSlope, zeroChain, height, min, shelfLength, sigmaMean, averagePercentile, signalToNoise, start, end, shelf_chiSq, shelf_p-value, recover_chiSq, recover_p-value, shelfDensity, trueShelf', file=open('{}/allShelves.csv'.format(dir1), 'a'))
#print('index, shelfID, shelfNumber, smoothing, min, max, difference, percentDifference, slope, zeroChain, length, sigmaMean, signalToNoise, start, end, quickTau, rawTau, trueTau', file=open('{}/tauShelves/shelves.csv'.format(dir1), 'a'))
print('index, smoothing, start, quickTau, rawTau, trueTau', file=open('{}/tauShelves/shelves.csv'.format(dir1), 'a'))

smoothings = [60, 40, 20, 10]

columns = {}
if fitsFile == 'glx_new_llsmocks2.fits':
    columns = {'id':'MockID', 
               'wave':'wave', 
               'flux':'flux', 
               'sigma':'sigma', 
               'zqso':'ZQSO', 
               'zlls':'ZLLS', 
               'tau':'TAU', 
               'fluxMod':1}
elif fitsFile == './gm16_sort_20200721.fits':
    columns = {'id':'sort_index', 
               'wave':'wave', 
               'flux':'flux', 
               'sigma':'sigma', 
               'zqso':'z', 
               'zlls':None, 
               'tau':None,
               'fluxMod':10**16}
    
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
accuracy = {}
guesses = 3
predictions = []
shelfChoice = []
tau = []
tauPreds = []
rawTau = []

printRange = []
for i in range(0, 20):
    printRange.append(int(i*len(mockSlice)/10))

correctShelfCount = 0

mainLock = multiprocessing.Lock()

i = 0
processList = []

coreCount = multiprocessing.cpu_count()
#coreCount = 1

tauModel = None
if makeTauModel:
    '''while i < len(mockSlice):
        for p in processList:
            if p.is_alive() == False:
                p.close()
                processList.remove(p)
        # Launch new processes
        while (len(processList) < coreCount) and (i < len(mockSlice)):
            if i in printRange:
                print('Working on', i)
            m = mockSlice[i]
            p = multiprocessing.Process(target=prepTauModel, args=(i, dir1, mockSlice, m, smoothings, mainLock))
            i += 1
            processList.append(p)
            p.start()        
    for p in processList:
        p.join()

    print('Training')
    fTau = dir1+'/tauShelves/shelves.csv'
    df = pd.read_csv(fTau)
    array = df.values
    X = array[:, 4:-1]
    y = array[:, -1]
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.2, random_state=1)
    tauModel = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    tauModel.fit(X_train, Y_train)
    print('Done training')'''

    for i in range(20):  
        time.sleep(random.random()) 
    sys.stdout.write("Please wait until I complete this feature ;)") 
if makeNewModel:
    i = 0
    while i < len(mockSlice):
        for p in processList:
            if p.is_alive() == False:
                p.close()
                processList.remove(p)
        # Launch new processes
        while (len(processList) < coreCount) and (i < len(mockSlice)):
            m = mockSlice[i]
            '''if i in printRange:
                print(datetime.now(), '--- Working on', i)'''
            currentDir = dir1 + '/definedShelves/match_{}.index'.format(m[columns['id']])
            os.mkdir(currentDir)
            with open(currentDir + '/matchInfo.txt', 'w') as thisFile:
                print(m, file=thisFile)
            p = multiprocessing.Process(target=prepModel, args=(i, dir1, currentDir, mockSlice, m, smoothings, mainLock, tauModel, columns))
            i += 1
            processList.append(p)
            p.start()
            sys.stdout.write('Running {}/{} {:6.2f}%  \r'.format(i, len(mockSlice), i/len(mockSlice)*100))
            sys.stdout.flush()
    for p in processList:
        p.join()
    
print('Done')

In [ ]:
if False:
    
    correct = 0
    higherTau = []
    highTau = []
    lowTau = []
    higherTauP = []
    highTauP = []
    lowTauP = []
    higherTauC = 0
    highTauCorrect = 0
    lowTauCorrect = 0
    for t in tau:
        if t > 0.5:
            if t > 1.5:
                higherTau.append(t)
            highTau.append(t)
        else:
            lowTau.append(t)
    for i, p in enumerate(predictions):
        if tau[i] > 0.5:
            if tau[i] > 1.5:
                higherTauP.append(p)
            highTauP.append(p)
        else:
            lowTauP.append(p)
        if abs(p) <= 200:
            correct += 1
            if tau[i] > 0.5:
                if tau[i] > 1.5:
                    higherTauC += 1
                highTauCorrect += 1
            else:
                lowTauCorrect += 1


    plt.hist2d(predictions, tau, (50, 50), cmap=plt.cm.jet)
    plt.xlabel('Distance in angstroms')
    plt.ylabel('TAU')
    plt.title('Distance vs tau')
    plt.colorbar()
    plt.show()
    plt.close()

    plt.hist2d(tauPreds, tau, (50, 50), cmap=plt.cm.jet)
    plt.xlabel('Estimated tau')
    plt.ylabel('TAU')
    plt.title('Estimated tau vs tau')
    plt.colorbar()
    plt.show()
    plt.close()
    print('TAU StDev         : {:.3g}'.format(statistics.stdev(tauPreds)))
    print('TAU Quantiles     : {}'.format(statistics.quantiles(tauPreds)))  
    print()
    #print('Correct Shelf  : {:.3g}'.format(correctShelf/len(shelfChoice)))
    print('Withing 200 AA    : {}/{} - {:.3g}%'.format(correct, len(predictions), correct/len(predictions)*100.0))
    print('StDev             : {:.3g}'.format(statistics.stdev(predictions)))
    print('Quantiles         : {}'.format(statistics.quantiles(predictions)))  
    print()
    print('Correct > 1.5 TAU : {}/{} - {:.3g}%'.format(higherTauC, len(higherTau), higherTauC/len(higherTau)*100.0))
    print('StDev             : {:.3g}'.format(statistics.stdev(higherTauP)))
    print('Quantiles         : {}'.format(statistics.quantiles(higherTauP)))
    print()
    print('Correct > 0.5 TAU : {}/{} - {:.3g}%'.format(highTauCorrect, len(highTau), highTauCorrect/len(highTau)*100.0))
    print('StDev             : {:.3g}'.format(statistics.stdev(highTauP)))
    print('Quantiles         : {}'.format(statistics.quantiles(highTauP)))
    print()
    print('Correct < 0.5 TAU : {}/{} - {:.3g}%'.format(lowTauCorrect, len(lowTau), lowTauCorrect/len(lowTau)*100.0))
    print('StDev             : {:.3g}'.format(statistics.stdev(lowTauP)))
    print('Quantiles         : {}'.format(statistics.quantiles(lowTauP)))
    print()
    print()
    print()
    print('Completed')

In [ ]:
# Chi^2
# Signal/noise
# Shelf length
# Min value
# Zero chain
# Shelf height
# Percentage shelf height
# Sigma
# Average slope

'''f = './automatedOutput/austinsMocks_modelSet_50000.fits/allShelves.csv'
df = pd.read_csv(f)
array = df.values
print(type(array))
X = array[:, 4:-1]
y = array[:, -1]
print(y)
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.2, random_state=1)'''

In [ ]:
# Spot Check Algorithms
if False:
    continuous = False
    models = []
    if continuous:
        models.append(('CART', DecisionTreeRegressor()))
        models.append(('ADABOOST', AdaBoostRegressor()))
    else:
        models.append(('Log Regress', LogisticRegression(solver='liblinear', multi_class='ovr')))
        models.append(('Linear Discrim', LinearDiscriminantAnalysis()))
        models.append(('K-Nearest', KNeighborsClassifier()))
        models.append(('Random Forest 100', RandomForestClassifier(n_estimators=100, n_jobs=-1)))
        #models.append(('Random Forest 200', RandomForestClassifier(n_estimators=200, n_jobs=-1)))
        #models.append(('Random Forest 500', RandomForestClassifier(n_estimators=500, n_jobs=-1)))
        models.append(('Ada Boost', AdaBoostClassifier()))
        models.append(('Bagging', BaggingClassifier()))
        models.append(('Extra Trees', ExtraTreesClassifier()))
        models.append(('Gradient Boost', GradientBoostingClassifier()))
        models.append(('MLP Neural', MLPClassifier()))
        #models.append(('NB', GaussianNB()))
        #models.append(('SVM', SVC(gamma='auto')))
    # evaluate each model in turn
    results = []
    names = []
    stackModels = []
    for name, model in models:
        stackModels.append(model)
        kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
        cv_results = cross_val_score(model, X_train, Y_train) # cv=kfold, scoring='accuracy')
        results.append(cv_results)
        names.append(name)
        print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
def returnPredictions(X_train, Y_train, X_validation, Y_validation):
    
    models = []
    models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
    models.append(('LDA', LinearDiscriminantAnalysis()))
    models.append(('KNN', KNeighborsClassifier()))
    models.append(('CART', DecisionTreeClassifier()))
    
    predictionsList = []
    for m in models:
        m = m[-1]
        m.fit(X_train, Y_train)
        predictionsList.append(m.predict(X_validation))
    predictions = []
    rawPredictions = []
    for i in range(0, len(predictionsList[0])):
        newPrediction = []
        for j in range(0, len(predictionsList)):
            newPrediction.append(predictionsList[j][i])
        rawPredictions.append(np.mean(newPrediction))
        predictions.append(int(np.mean(newPrediction)+0.5))
        
    return predictions, rawPredictions

In [ ]:
'''model = None
continuous = False

if continuous:
    model = DecisionTreeRegressor()
    model.fit(X_train, Y_train)
    predictions = model.predict(X_validation)
    print(r2_score(Y_validation, predictions))
    print(explained_variance_score(Y_validation, predictions))
    for p in predictions[:10]:
        print(predictions)
else:
    
    #predictions, rawPredictions = returnPredictions(X_train, Y_train, X_validation, Y_validation)
        
    #model = LogisticRegression(solver='liblinear', multi_class='ovr')
    model = DecisionTreeClassifier()
    model.fit(X_train, Y_train)
    predictions = model.predict(X_validation)
    
    print(accuracy_score(Y_validation, predictions))
    print(confusion_matrix(Y_validation, predictions))
    print(classification_report(Y_validation, predictions))
    
    scoring = {}
    scoring['TP'] = 0
    scoring['TN'] = 0
    scoring['FP'] = 0
    scoring['FN'] = 0
    for i in range(0, len(predictions)):
        if predictions[i] == Y_validation[i]:
            if predictions[i] == 1.:
                scoring['TP'] += 1
            else:
                scoring['TN'] += 1
        else:
            if predictions[i] == 1.:
                scoring['FP'] += 1
            else:
                scoring['FN'] += 1
    print(scoring)
    print('{}/{} - {:.3g}%'.format(scoring['TP']+scoring['TN'], scoring['TP']+scoring['TN']+scoring['FP']+scoring['FN'], 100*(scoring['TP']+scoring['TN'])/(scoring['TP']+scoring['TN']+scoring['FP']+scoring['FN'])))
    '''

In [23]:
import os, sys
import emcee
import corner
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.io import fits
from astropy.table import Table, unique, Column
import numpy as np
from linetools.spectra.xspectrum1d import XSpectrum1D
from pyigm.abssys.lls import LLSSystem
from pyigm.continuum import quasar as pyicq
from multiprocessing import Pool

def mcmodel1(theta1, pre_rbc):
    slope, scale = theta1

    xflux = slope * pre_rbc.wavelength.value + pre_rbc.flux * scale
    return xflux


def lnlike1(theta1, y1, yerr1, pre_rbc):
    LnLike = -0.5 * np.sum(((y1 - mcmodel1(theta1, pre_rbc)) / yerr1) ** 2)
    return LnLike


def lnprior1(theta1):
    slope, scale = theta1

    if scale > 0. and scale < 10000. and slope > -10. and slope < 10.:
        return 0.0
    return -np.inf


def lnprob1(theta1, y1, yerr1, pre_rbc):
    lp = lnprior1(theta1)

    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike1(theta1, y1, yerr1, pre_rbc)


def main1(p0, nwalkers, niter, ndim, lnprob1, data):
    #Pool for parallelizing
    with Pool() as pool:
        sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob1, args=data, pool=pool)

        #print("Running burn-in...")
        p0, _, _ = sampler.run_mcmc(p0, 100, progress=False)
        sampler.reset()

        #print("Running production...")
        pos, prob, state = sampler.run_mcmc(p0, niter, progress=False)
    return sampler, pos, prob, state


def plotter1(sampler, pre_rbc):
    plt.figure(figsize=(16, 10))
    plt.step(twl, tfl, c='k', alpha=.8)
    plt.step(wl, y1)
    samples = sampler.flatchain
    for theta in samples[np.random.randint(len(samples), size=100)]:
        plt.step(wl, mcmodel1(theta, pre_rbc), color="r", alpha=0.1)
    plt.xlim(1400, 2800)
    plt.ylim(-0.1, 5)
    plt.show()


def sample_walkers1(nsamples, flattened_chain, pre_rbc):
    models = []
    draw = np.floor(np.random.uniform(0, len(flattened_chain), size=nsamples)).astype(int)
    thetas = flattened_chain[draw]
    for i in thetas:
        mod = mcmodel1(i, pre_rbc)
        models.append(mod)
    spread = np.std(models, axis=0)
    med_model = np.median(models, axis=0)
    return med_model, spread

def mcmodel2(theta2, pre_rbc, slope, scale):
    zlls, tau = theta2
    nhi = 17.2 + np.log10(tau)

    # Create LLS Model
    lw = pre_rbc.wavelength.value
    lf = np.ones(len(lw))  # np.ones(len(lw))#+slope*lw
    lf1 = lf[np.where(lw < 912. * (1 + zlls))]
    lw1 = lw[np.where(lw < 912. * (1 + zlls))]
    lf2 = lf[np.where(lw >= 912. * (1 + zlls))]
    lf1x = lf1 * np.exp(-1. * tau * (lw1 / (912 * (1 + zlls))) ** 3)
    
    #
    model = XSpectrum1D.from_tuple((lw, np.concatenate((lf1x, lf2))))
    smmod = model.gauss_smooth(10)  # smooth abs. lines - this is generic, could make specific to GALEX
    lflux = smmod.flux
    xflux = lflux * (slope * lw + pre_rbc.flux * scale)  # /np.median(pre_rbc.flux[window])#normalize the LLS
    
    return xflux, zlls, tau


def lnlike2(theta2, y2, yerr2, pre_rbc, slope, scale):
    mcmod2, _, _ = mcmodel2(theta2, pre_rbc, slope, scale)
    LnLike = -0.5 * np.sum(((y2 - mcmod2) / yerr2) ** 2)
    return LnLike


def lnprior2(theta2, zMin, zMax):
    zlls, tau = theta2

    if zlls >= zMin and zlls <= zMax and tau > 0. and tau < 10:
        return 0.0
    return -np.inf


def lnprob2(theta2, y2, yerr2, pre_rbc, slope, scale, zMin, zMax):
    lp = lnprior2(theta2, zMin, zMax)

    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike2(theta2, y2, yerr2, pre_rbc, slope, scale)


def main2(p0, nwalkers, niter, ndim, lnprob2, data):
    with Pool() as pool:
        sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob2, args=data, pool=pool)

        #print("Running burn-in...")
        p0, _, _ = sampler.run_mcmc(p0, 100, progress=False)
        sampler.reset()

        #print("Running production...")
        pos, prob, state = sampler.run_mcmc(p0, niter, progress=False)

    return sampler, pos, prob, state


def plotter2(sampler):
    plt.figure(figsize=(16, 10))
    plt.step(wl, y2)
    samples = sampler.flatchain
    for theta in samples[np.random.randint(len(samples), size=100)]:
        mcmod2, _, _ = mcmodel2(theta2, pre_rbc, slope, scale)
        plt.step(twl, mcmod2, color="r", alpha=0.1)
    plt.xlim(1400, 2800)
    plt.ylim(-0.1, 5)
    plt.show()


def sample_walkers2(nsamples, flattened_chain, pre_rbc, slope, scale):
    models = []
    draw = np.floor(np.random.uniform(0, len(flattened_chain), size=nsamples)).astype(int)
    thetas = flattened_chain[draw]
    for i in thetas:
        mod, _, _ = mcmodel2(i, pre_rbc, slope, scale)
        models.append(mod)
    spread = np.std(models, axis=0)
    med_model = np.median(models, axis=0)
    return med_model, spread


In [24]:
def mcmcFit(zInit, tauInit, system, line, zReal, zMin, zMax, columns):

    # Change path to table if you run from somewhere else...
    #glx = Table.read('../gm16_sort_20200721.fits')

    #####
    ##
    ##  Set LLS Info (num: index from table, pre_zlls, pre_tau, pre_scale, pre_slope - guesses for pt I and II)
    ##
    #####
    num = system[columns['id']]
    zqso = system[columns['zqso']]
    norm = 2600
    pre_zlls = zInit
    pre_tau = tauInit
    pre_nhi = 17.2 + np.log10(pre_tau)
    pre_scale = 1.
    pre_slope = 0.

    twl = system[columns['wave']]
    tfl = system[columns['flux']]*columns['fluxMod']
    tel = system[columns['sigma']]*columns['fluxMod']

    tspec = XSpectrum1D.from_tuple((twl, tfl, tel))

    ######
    ##
    ## Part One: Fit continuum functions
    ##
    ##
    ######

    ####
    ##
    ## Fit continuum code
    ##
    ####

    # Set wavelength region for continuum fit (above LLS break)
    frange = np.where(twl > (1 + pre_zlls) * 800)
    wl = twl[frange]
    fl = tfl[frange]
    el = tel[frange]

    gspec = XSpectrum1D.from_tuple((wl, fl, el))

    # load continuum (Telfer)
    pre_cont = pyicq.get_telfer_spec(zqso)

    # Rebin continuum to GALEX
    pre_rbc = pre_cont.rebin(gspec.wavelength)

    y1 = gspec.flux
    yerr1 = gspec.sig

    data = (y1, yerr1, pre_rbc)

    # Adjust/increase nwalkers,niter if not testing
    nwalkers = 100
    niter = 500
    initial = np.array([pre_slope, pre_scale])
    ndim = len(initial)
    p0 = [np.array(initial) + 1e-7 * np.random.randn(ndim) for i in range(nwalkers)]

    sampler, pos, prob, state = main1(p0, nwalkers, niter, ndim, lnprob1, data)
    samples = sampler.flatchain

    med_model, spread = sample_walkers1(nwalkers, samples, pre_rbc)

    # Uncomment if you want to plot/save plotter1 output
    #plotter1(sampler)

    theta_max1 = samples[np.argmax(sampler.flatlnprobability)]
    best_fit_model1 = mcmodel1(theta_max1, pre_rbc)
    # Uncomment if you want to see the best-fit model
    #plt.figure(figsize=(16, 10))
    #plt.step(twl, tfl, c='k', alpha=.8)
    #plt.step(wl, y1)
    #plt.step(wl, best_fit_model1, c='r', label='Highest Likelihood Model')
    #plt.xlim(1400, 2800)
    #plt.ylim(-0.1, 5)
    #plt.legend()
    #plt.savefig('./mcmcOutput/test{}_bestFitModel1.png'.format(num))

    # Uncomment if you want the corner plots for the continuum fit
    #labels = ['slope', 'scale']
    #fig = corner.corner(samples, show_titles=True, labels=labels, plot_datapoints=True, quantiles=[0.16, 0.5, 0.84])

    # Create/save highest likelihood model with 1-sigma spread
    '''plt.figure(figsize=(16, 10))
    plt.step(twl, tfl, c='k', alpha=.8)
    plt.step(wl, y1)
    plt.step(wl, best_fit_model1, label='Highest Likelihood Model', c='r')
    plt.fill_between(wl, med_model - spread, med_model + spread, color='grey', alpha=0.5,
                     label=r'1$\sigma$ Posterior Spread')
    plt.xlim(1400, 2800)
    plt.ylim(-0.1, 5)
    plt.legend()
    plt.savefig('./mcmcOutput/test{}_cont.png'.format(num))'''

    # Save these values for the z,tau portion
    slope=theta_max1[0]
    scale=theta_max1[1]

    ######
    ##
    ## Part Two: Fit LLS
    ##
    ##
    ######

    ####
    ##
    ## Fit LLS code
    ##
    ####

    # Set continuum over entire GALEX spectrum
    pre_rbc = pre_cont.rebin(tspec.wavelength)

    y2 = tfl
    yerr2 = tel
    
    data = (y2, yerr2, pre_rbc, slope, scale, zMin, zMax)

    #Adjust/increase nwalkers,niter if not testing
    nwalkers = 100
    niter = 100
    initial = np.array([pre_zlls, pre_tau])
    ndim = len(initial)
    p0 = [np.array(initial) + 1e-7 * np.random.randn(ndim) for i in range(nwalkers)]

    sampler, pos, prob, state = main2(p0, nwalkers, niter, ndim, lnprob2, data)
    samples = sampler.flatchain

    med_model, spread = sample_walkers2(nwalkers, samples, pre_rbc, slope, scale)

    # Uncomment if you want to plot/save plotter2 output
    #plotter2(sampler)

    theta_max2 = samples[np.argmax(sampler.flatlnprobability)]
    best_fit_model2, zlls, mcmcTau = mcmodel2(theta_max2, pre_rbc, slope, scale)
    # Uncomment if you want to see the best-fit model
    #plt.figure(figsize=(16, 10))
    #plt.step(wl, y2)
    #plt.step(twl, best_fit_model2, label='Highest Likelihood Model')
    #plt.xlim(1400, 2800)
    #plt.ylim(-0.1, 5)
    #plt.legend()
    #plt.savefig('./mcmcOutput/test{}_bestFitModel2.png'.format(num))

    # Create/save corner plot
    plt.clf()
    labels = ['zlls', 'tau']
    fig = corner.corner(samples, show_titles=True, labels=labels, plot_datapoints=True, quantiles=[0.16, 0.5, 0.84])
    fig.savefig('./mcmcOutput/test{}_corner.png'.format(num))

    # Create/save highest likelihood model with 1-sigma spread
    plt.figure(figsize=(16, 10))
    plt.step(twl, y2)
    plt.step(twl, best_fit_model2, label='Highest Likelihood Model')
    plt.fill_between(twl, med_model - spread, med_model + spread, color='grey', alpha=0.5,
                     label=r'1$\sigma$ Posterior Spread')
    plt.vlines(912*(1+zInit), -0.1, 10, color='b', ls=':', lw=5, label='auto pred {}'.format(912*(1+zInit)))
    plt.vlines(912*(1+zlls), -0.1, 10, color='r', ls='--', lw=5, label='mcmc pred {}'.format(912*(1+zlls)))
    if zReal != None:
        plt.vlines(912*(1+zReal), -0.1, 10, color='g', lw=5, label='z real')
    plt.xlim(1400, 2800)
    plt.ylim(-0.1, 10)
    plt.legend()
    plt.savefig('./mcmcOutput/test{}_lls.png'.format(num))
    plt.close()
    
    return zlls, mcmcTau, best_fit_model2, twl

In [25]:
def makePrediction(f, zndex, mockSlice, m, smoothings, mainLock, model, columns, plot=False, matching=None):
    
    realLLS = None
    realTAU = None
    if columns['zlls'] != None:
        realLLS = m[columns['zlls']]
    if columns['tau'] != None:
        realTAU = m[columns['tau']]
        
    badPixels = len(m[columns['flux']][(m[columns['flux']] <= 0) & (m[columns['sigma']] < 0)])
        
    shelfFile = './automatedOutput/{}/definedShelves/match_{}.index/shelves.csv'.format(f, m[columns['id']])
    shelfDef = './automatedOutput/{}/definedShelves/match_{}.index/shelves_def.csv'.format(f, m[columns['id']])
    
    si = Table.read(shelfFile)
    sd = Table.read(shelfDef)
    
    mainWave = np.array(m[columns['wave']][(m[columns['wave']] >= 1400) & (m[columns['wave']] <= 2800)])
    mainFlux = np.array(m[columns['flux']][(m[columns['wave']] >= 1400) & (m[columns['wave']] <= 2800)])*columns['fluxMod']
    mainSigma = np.array(m[columns['sigma']][(m[columns['wave']] >= 1400) & (m[columns['wave']] <= 2800)])*columns['fluxMod']
    
    allShelves = []
    currentShelves = [list(range(0, len(mainWave)))]
    averageGuess = []
    tauEstimate = []
    newRawTau = []
    allSmoothings = []
    predictionDensity = [0.]*len(mainWave)
    rawDensity = [0.]*len(mainWave)
    tauFloor = [0.]*len(mainWave)
    tauRoof = [0.]*len(mainWave)
    tauPlot = [0.]*len(mainWave)
    longPredConf = [0.]*len(mainWave)
    peaks = []
    
    df = np.array(calculateEMA(getSigmaSmoothed(mainFlux, 5, mainSigma), 3, 12))
    ddx = [0.]*len(mainWave)
    dr = 5
    for i in range(1, len(mainWave)):
        ddx[i] += (df[i]-df[i-1])/(mainWave[i]-mainWave[i-1])*10
    ddx = getPureSmoothed(ddx, 5)
    
    for i in range(0, len(si)):
        if si[i]['smoothing'] in smoothings:
            if not si[i]['smoothing'] in allSmoothings:
                allSmoothings.append(si[i]['smoothing'])
            info = np.array(list(si[i].values())[3:]).reshape(1, -1)
            prediction = model.predict_proba(info)[0]
            mod = si[i]['smoothing']/50
            #mod = 1.0

            shelf = list(range(sd[i]['start'], sd[i]['start']+sd[i]['length']))
            for s in shelf:
                dichroicMod = 1.0
                '''if 1750 <= mainWave[s] <= 2050:
                    dichroicMod = 0.99'''
                predictionDensity[s] += prediction[1]*mod*dichroicMod
                rawDensity[s] += prediction[1]
                if si[i]['smoothing'] >= 40:
                    longPredConf[s] += prediction[1]*mod
                    
            # Tau predictions
            if si[i]['smoothing'] == 40:
                for s in shelf:
                    tauPlot[s] += si[i]['tauPrediction']
            if si[i]['smoothing'] == smoothings[0]:
                for s in shelf:
                    tauFloor[s] += si[i]['tauPrediction']
            if si[i]['smoothing'] == smoothings[-1]:
                for s in shelf:
                    tauRoof[s] += si[i]['tauPrediction']
                peaks.append((shelf[0], si[i]['start'], prediction[1]))
    
    peaks = sorted(peaks, key=(lambda x: -1*x[1]))
    
    mod = -29
    maxIndex = 0
    maxValue = 0.
    for i, p in enumerate(predictionDensity):
        if p > maxValue:
            maxIndex = i
            maxValue = p
            
    #maxValue = float(rawDensity[maxIndex])/float(len(allSmoothings))
    maxValue = float(rawDensity[maxIndex])
    wvp = mainWave[maxIndex]
    
    maxIndex = 0
    maxValue = 0.
    for i, p in enumerate(rawDensity):
        if p > maxValue:
            maxIndex = i
            maxValue = p
    longIndex = 0
    longMax = 0.
    for i, p in enumerate(longPredConf):
        if p > longMax:
            longIndex = i
            longMax = p
            
    newVP = (mainWave[longIndex]+mainWave[maxIndex])/2.
    
    wvp = mainWave[maxIndex]
    
    predTau = tauPlot[maxIndex]
    if len(si) == 0:
        wvp = 912*(1+m[columns['zqso']])
        newVP = 912*(1+m[columns['zqso']])
        
    if wvp > 912*(1+m[columns['zqso']]):
        wvp = 912*(1+m[columns['zqso']])
                
    add = 0.0112
    # Adjust for tau prediction
    if wvp != 912*(1+m[columns['zqso']]):
        if predTau >= 2.0:
            mod = add*912*-1
            wvp += mod
        elif predTau >= 0.5:
            mod = add*1.5*912*-1
            wvp += mod
        else:
            mod = add*3*912*-1
            wvp += mod
    
    shelfChoice = None
    for s in sd:
        if s['start'] <= maxIndex <= (s['start']+s['length']):
            if str(s['shelfID'])[0] == str(int(smoothings[0]/10)):
                shelfChoice = (mainWave[s['start']:s['start']+s['length']], np.array(allAboveValue(getSigmaSmoothed(mainFlux, 40, mainSigma), 0))[s['start']:s['start']+s['length']])
                break
    zMin = 0.5
    zMax = m[columns['zqso']]
    if shelfChoice != None:
        zMin = shelfChoice[0][0]/912. -1.
        zMax = shelfChoice[0][-1]/912. -1.
    
    #print(zMin, zMax)
    
    newZ = 0.
    devTau = 0.
    step = []
    stepWL = []
    #newZ, devTau, step, stepWL = mcmcFit((wvp/912.)-1., predTau, m, df, realLLS, zMin, zMax, columns)
    
    # Adjust for tau prediction
    if predTau >= 2.0:
        mod = 0.016377308081307433*-1.
        newZ += mod
    elif predTau >= 0.5:
        mod = 0.019471370402774513*-1.
        newZ += mod
    else:
        mod = 0.033917107411599326*-1.
        newZ += mod
        
    newVP = 912.*(1+newZ)
    
    if plot:
        yMax = max(allAboveValue(getSigmaSmoothed(mainFlux, 40, mainSigma), 0))*2
        if yMax < 5:
            yMax = 5
        figure(figsize=(16, 9))
        for wi in range(0, len(mainWave)):
            if mainFlux[wi] <= 0 and mainSigma[wi] < 0:
                plt.vlines(mainWave[wi], -1, yMax, lw=3, alpha=0.3, color='k')
        plt.plot(mainWave, mainFlux*0, color='k', ls=':')
        plt.plot(mainWave, mainFlux, alpha=0.3, label='raw flux')
        plt.plot(mainWave, mainSigma, alpha=0.3, color='orange', label='sigma')
        plt.plot(mainWave, allAboveValue(getSigmaSmoothed(mainFlux, 40, mainSigma), 0), label='smooth flux')
        #plt.plot(mainWave, allAboveValue(austinSmooth(mainFlux, 40, mainSigma), 0), color='green', label='austin flux')
        #plt.plot(mainWave, ddx, label='ddx', lw=3)
        #plt.plot(mainWave, predictionDensity, lw=3, label='confidence')
        plt.plot(mainWave, rawDensity, label='raw confidence')
        #plt.plot(mainWave, longPredConf, label='long confidence')
        plt.plot(mainWave, tauPlot, label='tau plot')
        #plt.plot(mainWave, tauFloor, ls='--', label='tau floor')
        #plt.plot(mainWave, tauRoof, ls = '--', label='tau roof')
        plt.vlines(wvp, -1, yMax, lw=5, ls='--', color='purple', label='auto prediction')
        #plt.vlines(newVP, -1, yMax, lw=5, ls='--', color='blue', label='mcmc prediction')
        if columns['zlls'] != None:
            plt.vlines(912*(1+m[columns['zlls']]), -1, yMax, lw=5, ls='--', color='green', label='actual LLS')
        plt.vlines(912*(1+m[columns['zqso']]), -1, yMax, lw=5, ls='--', color='red', label='zqso - {}'.format(912*(1+m[columns['zqso']])))
        if shelfChoice != None:
            plt.plot(shelfChoice[0], shelfChoice[1], color='purple', lw=6, label='shelfChoice')
        if len(step) > 0:
            plt.plot(stepWL, step, color='red', lw=3, label='mcmc shelf')
        for pi, p in enumerate(peaks):
            if rawDensity[p[0]] >= 2:
                plt.vlines(p[1], -1, yMax, lw=3, color='k', label='#{} - {:7.2f}'.format(pi+1, p[1]))
        plt.xlim(1400, 2800)
        plt.ylim(-1, yMax)
        plt.legend()
        plt.title('ID: {}, Index: {}, Real TAU: {}'.format(m[columns['id']], zndex, realTAU))
        plt.savefig('./automatedOutput/{}/{}.png'.format(f, m[columns['id']]))
        plt.close()
    
    printStr = '{},{},{},{},{},{},{},{},{},{},{},{},{}'.format(m[columns['id']], maxValue/len(smoothings), (wvp/912.)-1., predTau, (newVP/912.)-1., devTau, realLLS, realTAU, m[columns['sn']], m[columns['zqso']], badPixels, zMin, zMax)
    
    mainLock.acquire()
    try:
        print(printStr, file=open('./automatedOutput/{}/final.csv'.format(f), 'a'))
    finally:
        mainLock.release()
    
    return 0 #newVP-912*(1+m['ZLLS']), m['TAU']
        

In [74]:
def runModel(model, f, columns, plot, altFile=None):
    
    print('mockID,density,predZ,predTau,devZ,devTau,realZ,realTau,sn,zqso,badPixels,zrMin,zrMax', file=open('./automatedOutput/{}/final.csv'.format(f), 'w'))
    
    mocks = Table.read(f)
    matchFile = None
    if altFile != None:
        matchFile = Table.read(altFile)
    mockSlice = mocks
    printRange = []
    for i in range(0, 10):
        printRange.append(int(i*len(mockSlice)/10))

    predictions = []
    tau = []

    smoothings = [60, 40, 20, 10]

    mainLock = multiprocessing.Lock()

    i = 0
    processList = []
    coreCount = multiprocessing.cpu_count()
    
    while i < len(mockSlice):
        for p in processList:
            if p.is_alive() == False:
                p.close()
                processList.remove(p)
        # Launch new processes
        while (len(processList) < coreCount) and (i < len(mockSlice)):
            m = mockSlice[i]
            matching = None
            if altFile != None:
                matching = matchFile[matchFile['OBJID'] == m['objid']]
            p = multiprocessing.Process(target=makePrediction, args=(f, i, mockSlice, m, smoothings, mainLock, model, columns, plot, matching))
            i += 1
            sys.stdout.write('Running {}/{} {:6.2f}%  \r'.format(i, len(mockSlice), i/len(mockSlice)*100))
            sys.stdout.flush()
            processList.append(p)
            p.start()        
    for p in processList:
        p.join()
    

In [27]:
def autoPredictionSummary(info, idCol, predZCol, realZCol, predTauCol, realTauCol, snCol, conCol, title='', name=None, fname=None):
    
    if fname != None:
        print('mockID, predZ, realZ, gBound, predTau, realTau, snRed100, confidence, detectable', file=open(fname, 'w'))
    
    end = len(info)
    print(end)
    
    correct = 0
    correct100 = 0
    correct50 = 0
    gBoundPreds = []  # 20000 km/s separation
    higherTau = []
    highTau = []
    lowTau = []
    higherTauP = []
    highTauP = []
    lowTauP = []
    higherTauC = 0
    highTauCorrect = 0
    lowTauCorrect = 0
    
    predTauHold = []  # Copy of all predicted tau values for plotting later
    tauHold = []  # Copy of all tau values for plotting later
    tauOffset = []  # Copy of all z offset values for plotting later
            
    c = 300000
    
    for i in range(0, end):
        
        current = info[i]
        
        #print(i, current['mockID']) 
        
        thisID = current[idCol]
        thisPredZ = current[predZCol]
        thisRealZ = current[realZCol]
        thisPredTau = current[predTauCol]
        thisRealTau = current[realTauCol]
        thisSN = current[snCol]
        thisConfidence = current[conCol]
        
        zOffset = (thisPredZ-thisRealZ)/(1+thisRealZ)
        gBound = 0.067/(1+thisRealZ)
        gBoundPreds.append(zOffset)
        
        if thisRealTau >= 0.5:
            if thisRealTau >= 2.0:
                higherTauP.append(zOffset)
                higherTau.append(thisRealTau)
            highTauP.append(zOffset)
            highTau.append(thisRealTau)
        else:
            lowTauP.append(zOffset)
            lowTau.append(thisRealTau)
        
        predTauHold.append(thisPredTau)
        tauHold.append(thisRealTau)
        tauOffset.append(zOffset)
        
        if abs(zOffset) <= gBound*3:
            correct += 1
            if abs(zOffset) <= gBound*2:
                correct100 += 1
                if abs(zOffset) <= gBound*1:
                    correct50 += 1
                    if thisRealTau > 0.5:
                        if thisRealTau > 2.0:
                            higherTauC += 1
                        highTauCorrect += 1
                    else:
                        lowTauCorrect += 1
                 
        if fname != None:
            print('{}, {}, {}, {}, {}, {}, {}, {}, {}'.format(thisID, thisPredZ, thisRealZ, abs(zOffset) <= gBound, thisPredTau, thisRealTau, thisSN, thisConfidence, 'for Joe'), file=open(fname, 'a'))


    plt.hist2d(tauOffset, tauHold, (50, 50), cmap=plt.cm.jet)
    plt.xlabel('zOffset')
    plt.ylabel('TAU')
    plt.colorbar()
    plt.title(title)
    if name != None:
        plt.savefig(name)
    plt.show()
    plt.close()
    
    predictions = info
    
    if len(predictions) > 2:
        print('Within 60000 km/s    : {}/{} - {:.3g}%'.format(correct, len(predictions), correct/len(predictions)*100.0))
        print('Within 40000 km/s    : {}/{} - {:.3g}%'.format(correct100, len(predictions), correct100/len(predictions)*100.0))  
        print('Within 20000 km/s    : {}/{} - {:.3g}%'.format(correct50, len(predictions), correct50/len(predictions)*100.0))  
        print('StDev             : {}'.format(statistics.stdev(gBoundPreds)))
        print('Quantiles         : {}'.format(statistics.quantiles(gBoundPreds)))  
        print()
    if len(higherTau) > 2:
        print('Correct > 2.0 TAU : {}/{} - {:.3g}%'.format(higherTauC, len(higherTau), higherTauC/len(higherTau)*100.0))
        print('StDev             : {:.3g}'.format(statistics.stdev(higherTauP)))
        print('Quantiles         : {}'.format(statistics.quantiles(higherTauP)))
    print()
    if len(highTau) > 2:
        print('Correct > 0.5 TAU : {}/{} - {:.3g}%'.format(highTauCorrect, len(highTau), highTauCorrect/len(highTau)*100.0))
        print('StDev             : {:.3g}'.format(statistics.stdev(highTauP)))
        print('Quantiles         : {}'.format(statistics.quantiles(highTauP)))
    print()
    if len(lowTau) > 2:
        print('Correct < 0.5 TAU : {}/{} - {:.3g}%'.format(lowTauCorrect, len(lowTau), lowTauCorrect/len(lowTau)*100.0))
        print('StDev             : {:.3g}'.format(statistics.stdev(lowTauP)))
        print('Quantiles         : {}'.format(statistics.quantiles(lowTauP)))
    
    plt.hist2d(info[predTauCol], info[realTauCol], (50, 50), cmap=plt.cm.jet)
    plt.xlabel('Predicted tau')
    plt.ylabel('Real tau')
    plt.title('Pred tau vs real tau')
    plt.colorbar()
    plt.show()
    plt.close()

    plt.hist2d(tauOffset, predTauHold, (50, 50), cmap=plt.cm.jet)
    plt.xlabel('Distance in redshift')
    plt.ylabel('Predicted tau')
    plt.title('Z-offset vs predicted tau')
    plt.colorbar()
    plt.show()
    plt.close()
    
    print('---')
    print('---')
    print('---')
    print('Completed')
    

In [ ]:
def stackAlgo():
    
    stack = []
    stack.append(('Random Forest', RandomForestClassifier(n_estimators=100, n_jobs=-1)))
    stack.append(('Ada Boost', AdaBoostClassifier()))
    #stack.append(('Bagging', BaggingClassifier()))
    stack.append(('Extra Trees', ExtraTreesClassifier()))
    #stack.append(('Grad Boost', GradientBoostingClassifier()))
    #stack.append(('MLP Neural', MLPClassifier()))
    
    final = DecisionTreeClassifier()
    
    stackedAlgo = StackingClassifier(stack, final_estimator=final)
    
    return stackedAlgo


In [ ]:
f = './automatedOutput/austinsMocks_modelSet_50000.fits/allShelves.csv'
#f = './automatedOutput/glx_new_llsmocks2.fits/allShelves.csv'

df = pd.read_csv(f)
print(df.columns)
array = df.values

X_train = array[:, 4:-1]
Y_train = array[:, -1]

#X_train = array[:, :-1]
#Y_train = array[:, -1]
'''print(y)
X_train, X_validation, Y_train, Y_validation = train_test_split(X, y, test_size=0.1, random_state=1)
print(len(X_train))
print(len(Y_train))'''

In [ ]:
model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=None, min_samples_split=2, 
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                               max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, 
                               bootstrap=True, oob_score=False, n_jobs=-1, random_state=None, verbose=0, 
                               warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
print('Training')
model.fit(X_train, Y_train)
print('Done training')

pickle.dump(model, open('./models/shelfModel_50000.sav', 'wb'))

In [28]:
model = pickle.load(open('./models/shelfModel_50000.sav', 'rb'))
#name = 'austinsMocks_1000.fits'
#name = 'austinsMocks_25000.fits'
#name = 'glx_new_llsmocks2.fits'
name = 'gm16_sort_20200721.fits'
cols = list(Table.read('./automatedOutput/'+name+'/allShelves.csv').columns)
#cols = ['shelf_chiSq', 'shelfLength', 'averagePercentile']
fImps = []
for i in range(4, len(cols)-1):
    fImps.append((cols[i], model.feature_importances_[i-4]))
    
fImps = sorted(fImps, key=(lambda x: -1*x[-1]))

currentPercent = 0.
for imp in fImps:
    print(imp)
    currentPercent += imp[-1]

('averagePercentile', 0.1320373786948616)
('shelf_chiSq', 0.10887773417102081)
('shelfLength', 0.10393353456050883)
('end', 0.07418711143156884)
('min', 0.06985259546587619)
('height', 0.0650062530424391)
('start', 0.06244517297573254)
('zqso', 0.057153163570004654)
('percentHeight', 0.05634921351693638)
('sigmaMean', 0.05453328793653385)
('signalToNoise', 0.045951667417522234)
('averageSlope', 0.042371341087951375)
('shelf_p-value', 0.03934048288069316)
('recover_chiSq', 0.038619383986813194)
('shelfDensity', 0.024044564680445843)
('recover_p-value', 0.01136599978653558)
('smoothing', 0.009677139516579624)
('zeroChain', 0.004253975277976039)


In [29]:
columns = {}
if name == 'glx_new_llsmocks2.fits':
    columns = {'id':'MockID', 
               'wave':'wave', 
               'flux':'flux', 
               'sigma':'sigma', 
               'zqso':'ZQSO', 
               'zlls':'ZLLS', 
               'tau':'TAU',
               'sn':'S_N_red100',
               'fluxMod':1}
elif name == 'gm16_sort_20200721.fits':
    columns = {'id':'sort_index', 
               'wave':'wave', 
               'flux':'flux', 
               'sigma':'sigma', 
               'zqso':'z', 
               'zlls':None, 
               'tau':None,
               'sn':'SN_MEDIAN_ALL',
               'fluxMod':10**15}

f = './automatedOutput/{}/allShelves.csv'.format(name)
for f in glob.glob('./automatedOutput/{}/*.png'.format(name)):
    os.remove(f)
runModel(model, name, columns, True)

Process Process-1219:
Process Process-1218:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "<ipython-input-25-f50edb0d570f>", line 195, in makePrediction
    plt.savefig('./automatedOutput/{}/{}.png'.format(f, m[columns['id']]))
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/parallels/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-a04042cd696a>", line 26, in <module>
    runModel(model, name, columns, True)
  File "<ipython-input-26-dbc254b3a45a>", line 24, in runModel
    if p.is_alive() == False:
  File "/usr/lib/python3.8/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/parallels/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback 

Process Process-1217:
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/PIL/ImageFile.py", line 496, in _save
    fh = fp.fileno()


TypeError: object of type 'NoneType' has no len()

AttributeError: '_idat' object has no attribute 'fileno'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-25-f50edb0d570f>", line 195, in makePrediction
    plt.savefig('./automatedOutput/{}/{}.png'.format(f, m[columns['id']]))
  File "/home/parallels/.local/lib/python3.8/site-packages/matplotlib/pyplot.py", line 859, in savefig
    res = fig.savefig(*args, **kwargs)
  File "/home/parallels/.local/lib/python3.8/site-packages/matplotlib/figure.py", line 2311, in savefig
    self.canvas.print_figure(fname, **kwargs)
  File "/home/parallels/.local/lib/python3.8/site-packages/matplotlib/backend_bases.py", line 2210, in print_figure
    result = print_method(
  File "/home/parallels/.local/lib/python3.8

In [ ]:
name = 'glx_new_llsmocks2.fits'

print('--- --- --- ---')
print()
print('All mocks')
print('-')

f = './automatedOutput/{}/final.csv'.format(name)
ft = Table.read(f)

info = ft

autoPredictionSummary(info, 'mockID', 'predZ', 'realZ', 'predTau', 'realTau', 'snRed100', 'density', fname='./auto_fullMocks.csv')
autoPredictionSummary(info, 'mockID', 'devZ', 'realZ', 'devTau', 'realTau', 'snRed100', 'density', fname='./auto_fullMocks_developmentPreds.csv')

#id, tau, redshift, probability

In [ ]:
print('--- --- --- ---')
print()
print('No long zero chains')
print('-')

f = './automatedOutput/{}/final.csv'.format(name)
ft = Table.read(f)

info = ft[ft['zeroCount'] <= 15]

autoPredictionSummary(info, 'mockID', 'predZ', 'realZ', 'predTau', 'realTau', 'snRed100', 'density', fname='./auto_noZeroChains.csv')
autoPredictionSummary(info, 'mockID', 'devZ', 'realZ', 'devTau', 'realTau', 'snRed100', 'density', fname='./auto_noZeroChains_developmentPreds.csv')


In [ ]:
print('--- --- --- ---')
print()
print('All mocks')
print('-')

f = './automatedOutput/{}/final.csv'.format(name)
ft = Table.read(f)

info = ft[ft['realTau'] >= 0.5]

autoPredictionSummary(info, 'mockID', 'predZ', 'realZ', 'predTau', 'realTau', 'snRed100', 'density', fname='./auto_tauFloor.csv')
autoPredictionSummary(info, 'mockID', 'devZ', 'realZ', 'devTau', 'realTau', 'snRed100', 'density', fname='./auto_tauFloor_developmentPreds.csv')


In [ ]:
print('--- --- --- ---')
print()
print('Tau >= 2, SN >= 2')
print('-')

f = './automatedOutput/{}/final.csv'.format(name)
ft = Table.read(f)

info = ft[(ft['realTau'] >= 2) & (ft['snRed100'] >= 2)]

autoPredictionSummary(info, 'mockID', 'predZ', 'realZ', 'predTau', 'realTau', 'snRed100', 'density', fname='./auto_highTauSN.csv')
autoPredictionSummary(info, 'mockID', 'devZ', 'realZ', 'devTau', 'realTau', 'snRed100', 'density', fname='./auto_highTauSN_developmentPreds.csv')


In [ ]:
f = 'glx_new_llsmocks2.fits'
mocks = Table.read(f)

f = './automatedOutput/glx_new_llsmocks2.fits/final.csv'
ft = Table.read(f)

f = './glx_new_llsmocks2.fits'
fits = Table.read(f)
#print(fits)

allManualPreds = glob.glob('./manualPredictions/Lls/*')

names = []
manual = []

for m in allManualPreds:
    names.append(m)
    manual.append(Table.read(m, format='ascii'))

print('mockID, auto, autoTau, autoGBound, manVotes, manDeci, manual, manTau, manGBound, dev, devTau, devGBound, real, tau, zqso, snRed100', file=open('./comparison/compare.csv', 'w'))

end = max(manual[0]['mock_index'])

print(end)

for i in range(1, end+1):
    
    if len(ft[ft['mockID'] == i]) > 0:
        ftMatch = ft[ft['mockID'] == i][0]
        
    if len(fits[fits['MockID'] == i]):
        fitsMatch = fits[fits['MockID'] == i][0]
    
    figure(figsize=(32, 18))
    plt.plot(fitsMatch['wave'], fitsMatch['flux'], label='flux')
    plt.plot(fitsMatch['wave'], fitsMatch['flux']*0, color='k', ls=':')
    plt.plot(fitsMatch['wave'], fitsMatch['sigma'], label='sigma')
    plt.plot(fitsMatch['wave'], allAboveValue(getPureSmoothed(fitsMatch['flux'], 40), 0), lw=5, label='flux smoothed')
    plt.vlines(912*(1+fitsMatch['ZLLS']), -1, 5, lw=8, color='r', ls=':', label='LLS')
    
    for b in fitsMatch['wave'][(fitsMatch['flux'] <= 0) & (fitsMatch['sigma'] < 0)]:
        plt.vlines(b, -1, 5, lw=5, color='orange', alpha=0.5)
    
    #print(m['z_qso'], ftMatch['zqso'])
    con = []
    manDeci = 0.
    manVotes = 0
    manTau = []
    colors = ['r', 'b', 'g', 'orange', 'purple', 'pink', 'cyan', 'k', 'y']
    for man in manual:
        manVotes += 1
        allMarks = man[man['mock_index'] == i]
        if len(allMarks) == 1:  # currently only allowing single choices, double choices are not included in this output
            for marker, current in enumerate(allMarks):
                if current['cat'] != 'n':
                    manTau.append(current['tau'])
                    if current['cat'] == 'y':
                        plt.vlines(912.*(1+current['z_lls']), -1, 5, lw=5, color=colors[manVotes-1], label='{} #{}'.format(names[manVotes-1], marker))
                        manDeci += 1.
                    else:
                        plt.vlines(912.*(1+current['z_lls']), -1, 5, lw=5, ls='--', color=colors[manVotes-1], label='{} #{}'.format(names[manVotes-1], marker))
                        manDeci += 0.5
                    con.append(current['z_lls'])
    plt.vlines(912.*(1+ftMatch['predZ']), -1, 5, lw=8, color='k', alpha=0.7, label='auto')
    plt.vlines(912.*(1+fitsMatch['ZQSO']), -1, 5, lw=8, color='g', ls=':', label='zQSO')

    manualChoice = -999.
    manualDiff = -999.
    #print(con, ' --- ', 912*(1+fitsMatch['ZLLS']))
    
    autoGBound = False
    manGBound = False
    devGBound = False
    
    gBound = 0.067/(1+fitsMatch['ZLLS'])
    
    if len(con) > 0:
        manualChoice = np.median(con)
        manualDiff = manualChoice - ftMatch['realZ']
        manTau = np.mean(manTau)
        combined = (manualChoice+ftMatch['predZ'])/2 - ftMatch['realZ']
        manZ = manualChoice
        manZOffset = (manZ-fitsMatch['ZLLS'])/(1+fitsMatch['ZLLS'])
        if abs(manZOffset) <= gBound:
            manGBound = True
    else:
        manTau = -999.
        combined = (ftMatch['predZ']) - ftMatch['realZ']
    autoZ = ftMatch['predZ']
    autoZOffset = (autoZ-fitsMatch['ZLLS'])/(1+fitsMatch['ZLLS'])
    if abs(autoZOffset) <= gBound:
        autoGBound = True
        
    if abs((ftMatch['devZ']-fitsMatch['ZLLS'])/(1+fitsMatch['ZLLS'])) <= gBound:
        devGBound = True
    
    plt.ylim(-0.1, 5)
    plt.xlim(1400, 2800)
    plt.legend()
    plt.title(fitsMatch['MockID'])
    plt.savefig('./separateManualPredictions/{}.png'.format(fitsMatch['MockID']))
    plt.close()
    
    print('{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}'.format(ftMatch['mockID'], ftMatch['predZ'], ftMatch['predTau'], autoGBound, manVotes, manDeci, manualChoice, manTau, manGBound, ftMatch['devZ'], ftMatch['devTau'], devGBound, ftMatch['realZ'], ftMatch['realTau'], ftMatch['zqso'], fitsMatch['S_N_red100']), file=open('./comparison/compare.csv', 'a'))
    
    

In [ ]:
f = './comparison/compare.csv'
ft = Table.read(f)
#print(ft.columns)

'''for i in range(0, len(ft)):
    print(ft[i]['manIndex'], mocks[i]['MockID'])
    print(ft[i]['zqso'], mocks[i]['ZQSO'])
    print()'''

auto = ft['autoDiff']
man = ft['manualDiff']
real = ft['real']
tau = ft['tau']
autoTau = ft['autoTau']
manTau = ft['manTau']
autoWL = ft['auto']
manWL = ft['manual']

trueWL = []
trueAuto = []
trueMan = []
trueTau = []
trueAutoTau = []
trueManTau = []
combined = []
combinedTau = []
gBoundAuto = []
gBoundMan = []

for f in glob.glob('./comparison/mock_*.png'):
    os.remove(f)
for f in glob.glob('./comparison/badAutos/*.png'):
    os.remove(f)

for i, a in enumerate(ft):
    thisMock = mocks[mocks['MockID'] == a['autoIndex']][0]
    '''print(thisMock['ZQSO'], a['zqso'])
    print(thisMock['MockID'], a['autoIndex'])
    print()'''
    if ft[i]['manDeci'] >= 0.5 and thisMock['MockID'] == a['manIndex']:
        trueWL.append(a['real'])
        trueAuto.append(a['autoDiff'])
        if abs(a['autoDiff']) > 50:
            print(i, 'Correct' if abs(a['manualDiff']) <= 50 else 'Incorrect', a['manualDiff'])
        trueMan.append(a['manualDiff'])
        trueTau.append(a['tau'])
        trueAutoTau.append(a['autoTau'])
        trueManTau.append(a['manTau'])
        combined.append(a['combined'])
        combinedTau.append((a['autoTau']+a['manTau'])/2)
        
        gBoundAuto.append(a['auto'])
        gBoundMan.append(a['manual'])
        
        figure(figsize=(16, 9))
        plt.plot(mocks[i]['wave'], allAboveValue(calculateEMA(mocks[i]['flux'], 2, 12), 0))
        plt.vlines(ft[i]['manual'], -1, 5, lw=5, ls='--', color='red', label='manualMedian')
        plt.vlines(ft[i]['auto'], -1, 5, lw=5, ls='--', color='purple', label='autoPrediction')
        plt.vlines(ft[i]['real'], -1, 5, lw=5, color='green')
        plt.ylim(-0.1, 5)
        plt.xlim(1400, 2800)
        plt.legend()
        plt.title(thisMock['MockID'])
        plt.savefig('./comparison/{}_mock_{}.png'.format(a['autoTau'], thisMock['MockID']))
        plt.close()

#autoPredictionSummary(trueAuto, trueTau, gBoundAuto, trueWL, trueAutoTau, title='Auto prediction vs tau')
#autoPredictionSummary(trueMan, trueTau, gBoundMan, trueWL, trueManTau, title='Manual prediction vs tau', name='./manual_allMocks.png')
#printPredictionSummary(combined, trueTau, combinedTau, title='Combined vs tau', name='./comparison/aaa_combined.png')
#printPredictionSummary(auto, tau, autoTau, title='All auto', name='./comparison/aaa_ALL-auto.png')
#printPredictionSummary(man, tau, manTau, title='All manual', name='./comparison/aaa_ALL-manual.png')


In [ ]:
plt.hist2d(real, autoTau, (50, 50), cmap=plt.cm.jet)
plt.xlabel('Real location')
plt.ylabel('Distance')
plt.title('Distance vs location')
plt.colorbar()
plt.show()
plt.close()

In [ ]:
f = './comparison/compare.csv'
ft = Table.read(f)

algNames = ['auto', 'man', 'dev']

for n in algNames:

    xticks = [-1., 0.5, 1.0, 1.5, 2.0]
    xlabel = 'Signal-to-Noise'

    yticks = [0., 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
    ylabel = 'Real TAU'

    percentData = []
    correctCount = []
    totalCount = []
    tauSD = []
    avgZ = []

    for x in yticks:
        newData = []
        newCorrect = []
        newTotal = []
        newAvg = []
        newSD = []
        for y in xticks:
            section = ft[(ft['snRed100'] >= y) & (ft['tau'] >= x)]
            num = len(section[section['{}GBound'.format(n)] == 'True'])
            den = len(section)
            newData.append(num/den*100.)
            newCorrect.append(num)
            newTotal.append(den)
            newAvg.append((np.median(section['real'])))
            taus = []
            for ti in range(0, len(section['tau'])):
                if False:
                    newSD.append(np.std(section[section['{}Tau'.format(n)]-section['tau']]))
        percentData.append(newData)
        correctCount.append(newCorrect)
        totalCount.append(newTotal)
        avgZ.append(newAvg)

    percentData.reverse()
    correctCount.reverse()
    totalCount.reverse()
    avgZ.reverse()

    # signal-to-noise vs tau vs true positive

    title = 'Completeness (within 20000 km/s) for S/N vs TAU'

    flatten = lambda x: [item for sublist in x for item in sublist]
    flatList = flatten(percentData)
    fig, ax = plt.subplots()
    fig.set_size_inches(16, 16)
    im = ax.imshow(percentData, cmap=cm.coolwarm, norm=LogNorm(vmin=27, vmax=100))
    ax.set_xticks(np.arange(len(xticks)))
    ax.set_xticklabels(xticks)
    plt.xlabel(xlabel)
    ax.set_yticks(list(x for x in range(len(yticks)-1, -1, -1)))
    ax.set_yticklabels(yticks)
    plt.ylabel(ylabel)

    plt.title(title)

    for i in range(0, len(yticks)):
        for j in range(0, len(xticks)):
            text = ax.text(j, i, '{:.3g}%\n{}/{}\n<z> = {:.4g}\n'.format(percentData[i][j], correctCount[i][j], totalCount[i][j], avgZ[i][j]), ha='center', va='center', color='k')

    plt.show()

# wavelength vs tau vs false positive

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

xticks = [-1., 0.5, 1.0, 1.5, 2.0]
xlabel = 'Signal-to-Noise'

yticks = [0., 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
ylabel = 'Real TAU'

percentData = []
correctCount = []
totalCount = []
avgZ = []

for x in yticks:
    newData = []
    newCorrect = []
    newTotal = []
    newAvg = []
    for y in xticks:
        section = ft[(ft['snRed100'] >= y) & (ft['realTau'] >= x)]
        num = len(section[section['gBound'] == 'True'])
        den = len(section)
        newData.append(num/den*100.)
        newCorrect.append(num)
        newTotal.append(den)
        newAvg.append(list(section['realZ']))
    percentData.append(newData)
    correctCount.append(newCorrect)
    totalCount.append(newTotal)
    avgZ.append(newAvg)

percentData.reverse()
correctCount.reverse()
totalCount.reverse()
avgZ.reverse()

# signal-to-noise vs tau vs true positive

title = 'Completeness (within 20000 km/s) for S/N vs TAU'

flatten = lambda x: [item for sublist in x for item in sublist]
flatList = flatten(percentData)
fig, ax = plt.subplots()
fig.set_size_inches(16, 16)
im = ax.imshow(percentData, cmap=cm.coolwarm, norm=LogNorm(vmin=min(flatList), vmax=max(flatList)))
ax.set_xticks(np.arange(len(xticks)))
ax.set_xticklabels(xticks)
plt.xlabel(xlabel)
ax.set_yticks(list(x for x in range(len(yticks)-1, -1, -1)))
ax.set_yticklabels(yticks)
plt.ylabel(ylabel)

plt.title(title)

for i in range(0, len(yticks)):
    for j in range(0, len(xticks)):
        text = ax.text(j, i, '{:.3g}%\n{}/{}\n<z> = {:.4g}\n'.format(percentData[i][j], correctCount[i][j], totalCount[i][j], np.median(avgZ[i][j])), ha='center', va='center', color='k')

plt.show()

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

xticks = [-1, 0.5, 1.0, 1.5, 2.0]
xlabel = 'Signal-to-Noise'

yticks = [0.00, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
ylabel = 'Real TAU'

percentData = []
correctCount = []
totalCount = []

for x in yticks:
    newData = []
    newCorrect = []
    newTotal = []
    for y in xticks:
        section = ft[(ft['snRed100'] >= y) & (ft['realTau'] >= x) & (ft['confidence'] >= 0.9)]
        num = len(section[section['gBound'] == 'True'])
        den = len(section)
        newData.append(num/den*100.)
        newCorrect.append(num)
        newTotal.append(den)
    percentData.append(newData)
    correctCount.append(newCorrect)
    totalCount.append(newTotal)

percentData.reverse()
correctCount.reverse()
totalCount.reverse()

# signal-to-noise vs tau vs true positive

title = 'True positive (above 80% confidence, w/n 20000 km/s) for S/N vs TAU'

flatten = lambda x: [item for sublist in x for item in sublist]
flatList = flatten(percentData)
fig, ax = plt.subplots()
fig.set_size_inches(16, 16)
im = ax.imshow(percentData, cmap=cm.coolwarm, norm=LogNorm(vmin=27, vmax=max(flatList)))
ax.set_xticks(np.arange(len(xticks)))
ax.set_xticklabels(xticks)
plt.xlabel(xlabel)
ax.set_yticks(list(x for x in range(len(yticks)-1, -1, -1)))
ax.set_yticklabels(yticks)
plt.ylabel(ylabel)

plt.title(title)

for i in range(0, len(yticks)):
    for j in range(0, len(xticks)):
        text = ax.text(j, i, '{:.3g}%\n{}/{}'.format(percentData[i][j], correctCount[i][j], totalCount[i][j]), ha='center', va='center', color='k')

plt.show()

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

fp = ft[(ft['realTau'] <= 0.2) & (ft['gBound'] == 'False') & (ft['confidence'] >= 0.8)]

print(len(fp))

print(fp)

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

sn = [0., 0.5, 1.0, 1.5, 2.0]
taus = [0., 0.5, 1.0, 1.5, 2.0]
confs = [0., 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

fig, axs = plt.subplots(len(sn), len(taus))
fig.set_size_inches(40, 30)

fig.suptitle('Completeness vs Correctness of Automated Classification')

for si, s in enumerate(sn):
    
    for ti, t in enumerate(taus):

        total = ft[(ft['realTau'] >= t) & (ft['snRed100'] >= s)]

        comp = []
        corr = []

        for c in confs:

            section = total[total['confidence'] >= c]
            sectionCorrect = section[section['gBound'] == 'True']

            comp.append(len(section)/len(total)) 
            corr.append(len(sectionCorrect)/len(section))
        
        axs[si,ti].plot(confs, comp, label='Completeness')
        axs[si,ti].plot(confs, corr, label='Correctness')
        axs[si,ti].set_ylim(-0.1, 1.1)
        axs[si,ti].set_xlabel('Machine learning confidence score')
        axs[si,ti].set_ylabel('Percent')
        axs[si,ti].legend()
        axs[si,ti].set_title('Total systems: {}, Tau >= {}, S/N >= {}'.format(len(total), t, s))
            
        #plt.plot(confs, corr, label='Correctness')
        #plt.ylabel('Percent')
        #plt.legend()
        #plt.show()
        
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.3)
    

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

bins = list(range(1300, 3000, 50))

wlBins = []
accuracy = []
totals = []
corrects = []

figure(figsize=(16,9))

for i in range(0, len(bins)-1):
    
    total = ft[(bins[i] <= 912.*(1+ft['realZ'])) & (912.*(1+ft['realZ']) < bins[i+1])]
    
    if len(total) > 0:
        correct = total[total['gBound'] == 'True']
        wlBins.append('{}-{}'.format(bins[i], bins[i+1]))
        accuracy.append(len(correct)/len(total)*100)
        totals.append(len(total))
        corrects.append(len(correct))
        
plt.barh(wlBins, accuracy)
plt.ylabel('Wavelength bins')
plt.xlabel('Percent')
plt.title('Percent recovery per wavelength bin (1400-2800 in bins of 100 angstroms)')

for i, a in enumerate(accuracy):
    plt.text(a, i, '{:.3g}%\n{}/{}'.format(a, corrects[i], totals[i]), va='center')
    
plt.show()

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

bins = list(range(1300, 3000, 100))

wlBins = []
accuracy = []
totals = []
corrects = []

figure(figsize=(16,9))

for i in range(0, len(bins)-1):
    
    total = ft[(bins[i] <= 912.*(1+ft['realZ'])) & (912.*(1+ft['realZ']) < bins[i+1]) & (ft['confidence'] >= 0.8)]
    
    if len(total) > 0:
        correct = total[total['gBound'] == 'True']
        wlBins.append('{}-{}'.format(bins[i], bins[i+1]))
        accuracy.append(len(correct)/len(total)*100)
        totals.append(len(total))
        corrects.append(len(correct))
        
plt.barh(wlBins, accuracy)
plt.ylabel('Wavelength bins')
plt.xlabel('Percent')
plt.title('Percent recovery per wavelength bin (1400-2800 in bins of 100 angstroms) at >= 80% confidence')

for i, a in enumerate(accuracy):
    plt.text(a, i, '{:.3g}%\n{}/{}'.format(a, corrects[i], totals[i]), va='center')
    
plt.show()

In [ ]:
f = './auto_fullMocks.csv'
ft = Table.read(f)

bins = list(range(1300, 3000, 100))

print(bins)

confidence = []
accuracy = []

for i in range(0, len(bins)-1):
    
    total = ft[(bins[i] <= 912.*(1+ft['realZ'])) & (912.*(1+ft['realZ']) < bins[i+1])]
    
    if len(total) > 0:
        correct = total[total['gBound'] == 'True']
        wlBins.append(bins[i])
        accuracy.append(len(correct)/len(total))
        
plt.plot(wlBins, accuracy)

In [ ]:
def succinctFile(newFileName, dataTable, columnList, newHeaderNames=None):
    
    headers = ''
    for i, c in enumerate(columnList):
        if newHeaderNames == None:
            headers += '{},'.format(c)
        else:
            headers += '{},'.format(newHeaderNames[i])
    headers = headers[:-1]
    print(headers, file=open(newFileName, 'w'))
    
    allData = []
    for d in dataTable:
        newData = []
        for c in columnList:
            newData.append(d[c])
        allData.append(newData)
    
    allData = sorted(allData, key=(lambda x: x[0]))
    
    for d in allData:
        newLine = ''
        for c in d:
            newLine += '{},'.format(c)
        newLine = newLine[:-1]
        print(newLine, file=open(newFileName, 'a'))
        
    return 0


In [ ]:
f = './auto_fullMocks.csv'

newFileName = 'autoPredictions.csv'
dataTable = Table.read(f)
columnList = ['mockID', 'predZ', 'predTau', 'confidence', 'realZ']
newHeaderNames = ['mockID', 'autoZ', 'autoTau', 'autoConfidence', 'realZ']

succinctFile(newFileName, dataTable, columnList, newHeaderNames=newHeaderNames)

In [71]:
#f = './gm16_sort_20200721.fits'
f = 'output_mg2-0.5_lls-0.5.csv'

ft = Table.read(f)

ft

aID,sdssMatch,igMatch,lls,absPresent,absorber,absMatch,zMean,detectionZ,zDifference,specobjid,OBJID,plate,fiber,mjd,llsResponse,tauMean,absorberResponse,grade,zQSO
int64,str5,str5,str5,str5,str5,str5,float64,str7,str23,int64,str19,str5,str4,str5,float64,float64,str19,str2,str6
0,True,False,False,False,-,False,0.613,-,-,3388118830217698969,1237652946915098831,650,170,52143,0.2,3.6260000000000003,-,-,-
1,True,False,False,False,-,False,-99.9,-,-,3154740890152476590,1237663783124533374,686,131,52519,0.0,-99.9,-,-,-
2,True,True,False,False,Mg II,False,-99.9,0.29072,-,3155022365129183794,1237663543697080420,9406,495,58067,0.0,-99.9,0.0,C,0.6969
3,True,False,False,False,-,False,-99.9,-,-,3155022365129179746,1237657190906331255,9402,745,58039,0.0,-99.9,-,-,-
4,True,True,False,True,Mg II,False,-99.9,0.5499,-,3154740890152471724,1237657189832589394,9402,773,58039,0.0,-99.9,1.0,C,0.5549
5,True,False,False,False,-,False,-99.9,-,-,3155022365129185141,1237663543697145915,9407,244,58041,0.0,-99.9,-,-,-
6,True,True,False,True,Mg II,False,0.655,0.91679,-,3155022365129187208,1237663277928939566,9406,578,58067,0.1,0.32,1.0,C,1.4219
7,True,True,False,False,Mg II,False,0.655,0.32451,-,3155022365129187208,1237663277928939566,9406,578,58067,0.1,0.32,0.0,C,1.4219
8,True,False,False,False,-,False,-99.9,-,-,3155022365129189774,1237657191980269751,9407,254,58041,0.0,-99.9,-,-,-


In [73]:
f = './gm16_sort_20200721.fits'
#f = 'output_mg2-0.5_lls-0.5.csv'

ft = Table.read(f)

ft

sort_index,objid,specobjid,ra,dec,nuv_mag,fuv_mag,nuv_median_s2n,fuv_median_s2n,angsep_1,ra_index,RA_1,DEC_1,Target Name,Desc,Rmag,Bmag,Comment,Rs,Bs,z,Cite,zCite,Qpct,XID,RID,LID1,LID2,SDSS_NAME,RA_2,DEC_2,PLATE,MJD,FIBERID,AUTOCLASS_PQN,AUTOCLASS_DR14Q,IS_QSO_QN,Z_QN,RANDOM_SELECT,Z_10K,Z_CONF_10K,PIPE_CORR_10K,IS_QSO_10K,THING_ID,Z_VI,Z_CONF,CLASS_PERSON,Z_DR12Q,IS_QSO_DR12Q,Z_DR7Q_SCH,IS_QSO_DR7Q,Z_DR6Q_HW,Z_DR7Q_HW,IS_QSO_FINAL,Z,SOURCE_Z,Z_PIPE,ZWARNING,OBJID,Z_PCA,ZWARN_PCA,DELTACHI2_PCA,Z_HALPHA,ZWARN_HALPHA,DELTACHI2_HALPHA,Z_HBETA,ZWARN_HBETA,DELTACHI2_HBETA,Z_MGII,ZWARN_MGII,DELTACHI2_MGII,Z_CIII,ZWARN_CIII,DELTACHI2_CIII,Z_CIV,ZWARN_CIV,DELTACHI2_CIV,Z_LYA,ZWARN_LYA,DELTACHI2_LYA,Z_LYAWG,Z_DLA [5],NHI_DLA [5],CONF_DLA [5],BAL_PROB,BI_CIV,ERR_BI_CIV,AI_CIV,ERR_AI_CIV,BI_SIIV,ERR_BI_SIIV,AI_SIIV,ERR_AI_SIIV,BOSS_TARGET1,EBOSS_TARGET0,EBOSS_TARGET1,EBOSS_TARGET2,ANCILLARY_TARGET1,ANCILLARY_TARGET2,NSPEC_SDSS,NSPEC_BOSS,NSPEC,PLATE_DUPLICATE [74],MJD_DUPLICATE [74],FIBERID_DUPLICATE [74],SPECTRO_DUPLICATE [74],SKYVERSION,RUN_NUMBER,RERUN_NUMBER,CAMCOL_NUMBER,FIELD_NUMBER,ID_NUMBER,LAMBDA_EFF,ZOFFSET,XFOCAL,YFOCAL,CHUNK,TILE,PLATESN2,PSFFLUX [5],PSFFLUX_IVAR [5],PSFMAG [5],PSFMAGERR [5],EXTINCTION [5],M_I,SN_MEDIAN_ALL,GALEX_MATCHED,FUV,FUV_IVAR,NUV,NUV_IVAR,UKIDSS_MATCHED,YFLUX,YFLUX_ERR,JFLUX,JFLUX_ERR,HFLUX,HFLUX_ERR,KFLUX,KFLUX_ERR,W1_FLUX,W1_FLUX_IVAR,W1_MAG,W1_MAG_ERR,W1_CHI2,W1_FLUX_SNR,W1_SRC_FRAC,W1_EXT_FLUX,W1_EXT_FRAC,W1_NPIX,W2_FLUX,W2_FLUX_IVAR,W2_MAG,W2_MAG_ERR,W2_CHI2,W2_FLUX_SNR,W2_SRC_FRAC,W2_EXT_FLUX,W2_EXT_FRAC,W2_NPIX,FIRST_MATCHED,FIRST_FLUX,FIRST_SNR,SDSS2FIRST_SEP,JMAG,JMAG_ERR,JSNR,JRDFLAG,HMAG,HMAG_ERR,HSNR,HRDFLAG,KMAG,KMAG_ERR,KSNR,KRDFLAG,SDSS2MASS_SEP,2RXS_ID,2RXS_RA,2RXS_DEC,2RXS_SRC_FLUX,2RXS_SRC_FLUX_ERR,SDSS2ROSAT_SEP,XMM_SRC_ID,XMM_RA,XMM_DEC,XMM_SOFT_FLUX,XMM_SOFT_FLUX_ERR,XMM_HARD_FLUX,XMM_HARD_FLUX_ERR,XMM_TOTAL_FLUX,XMM_TOTAL_FLUX_ERR,XMM_TOTAL_LUM,SDSS2XMM_SEP,GAIA_MATCHED,GAIA_DESIGNATION,GAIA_RA,GAIA_DEC,GAIA_PARALLAX,GAIA_PARALLAX_ERR,GAIA_PM_RA,GAIA_PM_RA_ERR,GAIA_PM_DEC,GAIA_PM_DEC_ERR,GAIA_G_MAG,GAIA_G_FLUX_SNR,GAIA_BP_MAG,GAIA_BP_FLUX_SNR,GAIA_RP_MAG,GAIA_RP_FLUX_SNR,SDSS2GAIA_SEP,angsep_2,wave [486],flux [486],sigma [486]
int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,bytes25,bytes4,float64,float64,bytes2,bytes1,bytes3,float64,bytes6,bytes6,int64,bytes24,bytes21,bytes24,bytes21,bytes18,float64,float64,int32,int32,int16,bytes6,bytes6,int16,float64,int16,float64,int16,int16,int16,int64,float64,int16,int16,float64,int16,float64,int16,float64,float64,int16,float64,bytes12,float64,int32,bytes19,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float64,int64,float64,float32,float64,float64,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int32,int32,int32,int32,int32,int16,int16,uint8,int32,bytes3,int32,int32,int32,float64,float64,float64,float64,bytes14,int32,float64,float32,float64,float32,float64,float32,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,int32,float64,float64,float64,int32,float64,bytes21,float64,float64,float32,float32,float64,int64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float64,int16,bytes28,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
0,3388116631194443417,3388118830217698969,0.148870500897942,-10.5512608538128,19.49489974975586,20.882999420166016,0.8427219986915588,0.20239800214767456,0.00024755164605991486,0,0.148979

In [ ]:
# wave
# flux
# sigma

#218, 887

# Functionality to either build model or just find shelves
# Going to need a different model for sdss